# GPU Bootcamp for OpenACC/OpenMP

Welcome to the KSC 22 GPU bootcamp! This notebook will introduce you Nways to Accelerate GPU.

<img src="https://www.nvidia.com/content/dam/en-zz/Solutions/about-nvidia/logo-and-brand/02-nvidia-logo-color-wht-500x200-4c25-d@2x.png" height="100"></td></tr>
</table>



## DevOps - install NVIDIA HPC SDK
 


### step1. enabling GPU Support and check CUDA driva and CUDA toolkit in Google Colab. 

To use GPU resources through Colab, change the runtime to GPU:

 1. From the "Runtime" menu select "Change Runtime Type"
 2. Choose "GPU" from the drop-down menu
 3. Click "SAVE"
This will reset the notebook and probably ask you if you are a robot (these instructions assume you are not). Running

`!nvidia-smi`

in a cell will verify this has worked and show you what kind of hardware you have access to.

In [ ]:
# OS version. Probably Ubuntu 18.04 LTS.
! cat /etc/os-release

# GPU information
!nvidia-smi
!nvcc --version
!which nvcc

NAME="Ubuntu"
VERSION="18.04.6 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.6 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic
Tue Sep 20 04:06:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2

 ### step2. install NVIDIA HPC SDK 

 Let's install GPU compilers to the virtual machine. Following https://developer.nvidia.com/nvidia-hpc-sdk-downloads, we download and install debian packages. It will take some time( 7 minutes). It depends on network speed.

as you see in nvidia-smi log, 
 - 460.32.03 CUDA driver for CUDA toolkit 11.2 was installed in Google Colab.
 - CUDA toolkit 11.1 was installed in colab

So we will uninstall old cuda toolkit and install NVIDIA HPC SDK 22-7 with CUDA version 11.7. It takes 1 minutes to uninstall and 6 minutes to install NVIDIA HPC SDK 22.7 with CUDA toolkit 11.7.

<b> Caution !!! - after restarting the colab, you need to re-install the SDK because factory reset. It will take 7 minutes. </b>

for KISTI Neuron user,
you can install NVIDIA HPC SDK on Neuron system with below script

```
$ mkdir -p /scratch/$USER/temp
$ module load singularity/3.9.7
$ export SINGULARITY_CACHEDIR=/scratch/$USER/temp
$ cd /scratch/$USER
$ singularity build --sandbox nvhpc_2203_116_centos7.sid docker://nvcr.io/nvidia/nvhpc:22.3-devel-cuda11.6-centos7
$ cd /scratch/$USER/workdir
$ singularity run --nv /scratch/$USER/nvhpc_2203_116_centos7.sid bash 
```

In [ ]:
%%time 
# remove cuda toolkit
!echo "remove nvidia modules"
!time apt autoremove nvidia-*  > /etc/null
!echo "remove cuda toolkit"
!time apt autoremove cuda-*   > /etc/null
# install NVIDIA HPC SDK
!echo "update CUDA repository"
! echo 'deb [trusted=yes] https://developer.download.nvidia.com/hpc-sdk/ubuntu/amd64 /' | sudo tee /etc/apt/sources.list.d/nvhpc.list
!time apt-get update -y > /etc/null 
!echo "install NVIDIA HPC SDK"
!time apt-get install -y nvhpc-22-7 


remove nvidia modules



real	0m26.679s
user	0m4.308s
sys	0m5.443s
remove cuda toolkit



real	0m22.064s
user	0m13.461s
sys	0m3.054s
update CUDA repository
deb [trusted=yes] https://developer.download.nvidia.com/hpc-sdk/ubuntu/amd64 /
W: An error occurred during the signature verification. The repository is not updated and the previous index files will be used. GPG error: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease: The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
W: Failed to fetch https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/InRelease  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
W: Some index files failed to download. They have been ignored, or old ones used instead.

real	0m7.591s
user	0m4.027s
sys	0m1.509s
install NVIDIA HPC SDK
Reading package lists... Done
Building dependency 

### step3. configure environment variables
We need to configure  **PATH** and **LD_LIBRARY_PATH** variables. Thease files was installed as below : 
 - compiler binary : `/opt/nvidia/hpc_sdk/Linux_x86_64/2022/compilers/bin/` 
 - HPC SDK library : `/opt/nvidia/hpc_sdk/Linux_x86_64/2022/compilers/lib`
 - CUDA library : `/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/cuda/11.7/lib64`
 . 


In [ ]:
# Compiler version
! /opt/nvidia/hpc_sdk/Linux_x86_64/2022/compilers/bin/nvcc --version
! /opt/nvidia/hpc_sdk/Linux_x86_64/2022/compilers/bin/nvfortran --version
! /opt/nvidia/hpc_sdk/Linux_x86_64/2022/compilers/bin/nvc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0

nvfortran 22.7-0 64-bit target on x86-64 Linux -tp skylake-avx512 
NVIDIA Compilers and Tools
Copyright (c) 2022, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.

nvc 22.7-0 64-bit target on x86-64 Linux -tp skylake-avx512 
NVIDIA Compilers and Tools
Copyright (c) 2022, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.


### check and configure cuda toolkit

 We need to configure right location for CUDA and NVIDIA HPC SDK libraries and binary path.

In [ ]:
# default for colab
!echo $LD_LIBRARY_PATH
!echo $PATH

/usr/lib64-nvidia
/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin


In [ ]:
%env LD_LIBRARY_PATH=/opt/nvidia/hpc_sdk/Linux_x86_64/2022/compilers/lib:/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/cuda/11.7/lib64:/usr/lib64-nvidia:/usr/lib	
%env PATH=/opt/bin:/opt/nvidia/hpc_sdk/Linux_x86_64/2022/compilers/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bi

env: LD_LIBRARY_PATH=/opt/nvidia/hpc_sdk/Linux_x86_64/2022/compilers/lib:/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/cuda/11.7/lib64:/usr/lib64-nvidia:/usr/lib
env: PATH=/opt/bin:/opt/nvidia/hpc_sdk/Linux_x86_64/2022/compilers/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bi


In [ ]:
!nvidia-smi
!dmesg | grep NVRM
!nvcc --version
!nvfortran --version
!nvc --version

Tue Sep 20 04:26:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### step4. evalute toolkit

current situation
- cuda driver : 460.x ( cuda 11.2)
- cuda toolkit : cuda toolkit 11.7  ( need 515.x)
- nvidia HPC sdk : 22.7 for cuda toolkit 11.7 
              
https://docs.nvidia.com/deploy/cuda-compatibility/ 

**compile simple C code**

pi_serial.c 
 - `%%file ` command is save command in jupyter.
 - simple pi calculation with integral 

<center>
$  \displaystyle  \frac{\pi} {4} =\arctan(1) =   \int\limits_0^1 \! \frac{1}{1+ x^{2}}\, dx  $  
</center>

In [ ]:
%%file pi_serial.c

#include <stdio.h> 
 
main() {

 double pi = 0.0; 
 long i;
 long N = 4000000000 ; 

 for (i=0; i<N; i++) {
	 double t = (double)((i+0.05)/N);
	 pi += 4.0/(1.0+t*t);
 }
 printf("pi = %f with N %d \n", pi/N, N);
}

Writing pi_serial.c


In [ ]:
!nvc pi_serial.c -o a.out_serial && time  ./a.out_serial

pi = 3.141593 with N -294967296 

real	0m12.414s
user	0m12.345s
sys	0m0.010s


simple GPU acceleration with OpenACC

In [ ]:
%%file pi_acc.c

#include <stdio.h> 

main() {

 double pi = 0.0; 
 long i;
 long N = 400000000 ; 

#pragma acc data copy(pi)
#pragma acc parallel
#pragma acc for 
 for (i=0; i<N; i++) {
	 double t = (double)((i+0.05)/N);
	 pi += 4.0/(1.0+t*t);
 }
 printf("pi = %f with N %d \n", pi/N, N);
}

Writing pi_acc.c


In [ ]:
!nvc -acc -Minfo=accel  -ta=tesla pi_acc.c -o a.out_acc && time  ./a.out_acc

main:
     13, Generating copy(pi) [if not already present]
         Loop is parallelizable
         Generating NVIDIA GPU code
         13, #pragma acc loop vector(128) /* threadIdx.x */
             Generating implicit reduction(+:pi)
pi = 3.141593 with N 400000000 

real	0m0.791s
user	0m0.585s
sys	0m0.166s


compare result : 
 - CPU : 12 sec  single core(virtual) : 
 - GPU : 0.5 sec ( depends on colab GPU)

# Chapter 1  OpenACC

<img src="https://www.nvidia.com/content/dam/en-zz/Solutions/about-nvidia/logo-and-brand/02-nvidia-logo-color-wht-500x200-4c25-d@2x.png" height="100"></td></tr>
</table>

## OpenACC in Fortran


Let's execute the cell below to display information about the GPUs running on the server by running the nvaccelinfo command, which ships with the NVIDIA HPC compiler that we will be using. To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or pressing the play button in the toolbar above. If all goes well, you should see some output returned below the grey cell.

In [ ]:
!nvaccelinfo


CUDA Driver Version:           11020
NVRM version:                  NVIDIA UNIX x86_64 Kernel Module  460.32.03  Sun Dec 27 19:00:34 UTC 2020

Device Number:                 0
Device Name:                   Tesla V100-SXM2-16GB
Device Revision Number:        7.0
Global Memory Size:            16945512448
Number of Multiprocessors:     80
Concurrent Copy and Execution: Yes
Total Constant Memory:         65536
Total Shared Memory per Block: 49152
Registers per Block:           65536
Warp Size:                     32
Maximum Threads per Block:     1024
Maximum Block Dimensions:      1024, 1024, 64
Maximum Grid Dimensions:       2147483647 x 65535 x 65535
Maximum Memory Pitch:          2147483647B
Texture Alignment:             512B
Clock Rate:                    1530 MHz
Execution Timeout:             No
Integrated Device:             No
Can Map Host Memory:           Yes
Compute Mode:                  default
Concurrent Kernels:            Yes
ECC Enabled:                   Yes
Memory C

This gives us lots of details about the GPU, for instance the device number, the type of device, and at the very bottom the command line argument we should use when targeting this GPU (see NVIDIA HPC Compiler Option). We will use this command line option a bit later to build for our GPU.

# Introduction
Our goal for this lab is to learn the first steps in accelerating an application with OpenACC. We advocate the following 3-step development cycle for OpenACC.

<center> OpenACC development cycle

<img src="https://github.com/openhackathons-org/gpubootcamp/raw/5c077a6c70989492f4bd850240fdcaf8b16fd555/hpc/openacc/English/Fortran/jupyter_notebook/images/development-cycle.png" width=600> </center>

- **Analyze** your code to determine most likely places needing parallelization or optimization.

- **Parallelize** your code by starting with the most time consuming parts and check for correctness.

- **Optimize** your code to improve observed speed-up from parallelization.

One should generally start the process at the top with the analyze step. For complex applications, it's useful to have a profiling tool available to learn where your application is spending its execution time and to focus your efforts there. The NVIDIA HPC Software Development Kit (SDK) comes with NVIDIA Nsight Systems/Compute for interactive HPC applications performance profiler. Since our example code is quite a bit simpler than a full application, we'll skip profiling the code and simply analyze the code by reading it.

# Guide for the LAB

- For each lab, we will make sub directories. 

- We will use cell magic `%%file filename` to save source file. 
- you can save the file you can edit source code in the cell.  
- after writing, you can browse file in file expolore 

# Lab 1 

In [ ]:
!mkdir -p /content/lab01

In [ ]:
cd /content/lab01

/content/lab01


### prepare lab1 in colab

#### baseline code and makefile
 - lab01/Makefile
 - lab01/jacobi.f90  : main program 
 - lab01/laplace2d.f90 : subroutines 


In [ ]:
%%file Makefile
FC := nvfortran
ACCFLAGS_1 := -fast
ACCFLAGS_2 := -fast -acc -ta=multicore -Minfo=accel
ACCFLAGS_3 := -fast -acc -ta=tesla:managed -Minfo=accel

laplace_serial: laplace2d.f90 jacobi.f90 
	${FC} ${ACCFLAGS_1} -o laplace_serial laplace2d.f90 jacobi.f90

laplace_multicore: laplace2d.f90 jacobi.f90 
	${FC} ${ACCFLAGS_2} -o laplace_multicore laplace2d.f90 jacobi.f90 

laplace_gpu: laplace2d.f90 jacobi.f90 
	${FC} ${ACCFLAGS_3} -o laplace_gpu laplace2d.f90 jacobi.f90 

clean:
	rm -f *.o laplace_serial laplace_multicore laplace_gpu

Overwriting Makefile


In [ ]:
%%file jacobi.f90

program jacobi
  use laplace2d
  implicit none
  integer, parameter :: fp_kind=kind(1.0d0)
  integer, parameter :: n=4096, m=4096, iter_max=1000
  integer :: i, j, iter
  real(fp_kind), dimension (:,:), allocatable :: A, Anew
  real(fp_kind) :: tol=1.0e-6_fp_kind, error=1.0_fp_kind
  real(fp_kind) :: start_time, stop_time

  call initialize(A, Anew, m, n)
   
  write(*,'(a,i5,a,i5,a)') 'Jacobi relaxation Calculation:', n, ' x', m, ' mesh'
 
  call cpu_time(start_time) 

  iter=0
  
  do while ( error .gt. tol .and. iter .lt. iter_max )

    error = calcNext(A, Anew, m, n)
    call swap(A, Anew, m, n)

    if(mod(iter,100).eq.0 ) write(*,'(i5,f10.6)'), iter, error

    iter = iter + 1

  end do

  call cpu_time(stop_time) 
  write(*,'(a,f10.3,a)')  ' completed in ', stop_time-start_time, ' seconds'

  deallocate (A,Anew)
end program jacobi

Overwriting jacobi.f90


In [ ]:
%%file laplace2d.f90

module laplace2d
  public :: initialize
  public :: calcNext
  public :: swap
  public :: dealloc
  contains
    subroutine initialize(A, Anew, m, n)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(out)   :: A(:,:)
      real(fp_kind),allocatable,intent(out)   :: Anew(:,:)
      integer,intent(in)          :: m, n

      allocate ( A(0:n-1,0:m-1), Anew(0:n-1,0:m-1) )

      A    = 0.0_fp_kind
      Anew = 0.0_fp_kind

      A(0,:)    = 1.0_fp_kind
      Anew(0,:) = 1.0_fp_kind
    end subroutine initialize

    function calcNext(A, Anew, m, n)
      integer, parameter          :: fp_kind=kind(1.0d0)
      real(fp_kind),intent(inout) :: A(0:n-1,0:m-1)
      real(fp_kind),intent(inout) :: Anew(0:n-1,0:m-1)
      integer,intent(in)          :: m, n
      integer                     :: i, j
      real(fp_kind)               :: error

      error=0.0_fp_kind

      do j=1,m-2
        do i=1,n-2
          Anew(i,j) = 0.25_fp_kind * ( A(i+1,j  ) + A(i-1,j  ) + &
                                       A(i  ,j-1) + A(i  ,j+1) )
          error = max( error, abs(Anew(i,j)-A(i,j)) )
        end do
      end do
      calcNext = error
    end function calcNext

    subroutine swap(A, Anew, m, n)
      integer, parameter        :: fp_kind=kind(1.0d0)
      real(fp_kind),intent(out) :: A(0:n-1,0:m-1)
      real(fp_kind),intent(in)  :: Anew(0:n-1,0:m-1)
      integer,intent(in)        :: m, n
      integer                   :: i, j

      do j=1,m-2
        do i=1,n-2
          A(i,j) = Anew(i,j)
        end do
      end do
    end subroutine swap

    subroutine dealloc(A, Anew)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(in) :: A
      real(fp_kind),allocatable,intent(in) :: Anew

      deallocate (A,Anew)
    end subroutine
end module laplace2d

Overwriting laplace2d.f90


# Analyze the Code
In the section below you will learn about the algorithm implemented in the example code and see examples pulled out of the source code. If you want a sneak peek at the source code, you can take a look at the files linked below or open the downloaded file.

# code Description
The code simulates heat distribution across a 2-dimensional metal plate. In the beginning, the plate will be unheated, meaning that the entire plate will be room temperature. A constant heat will be applied to the edge of the plate and the code will simulate that heat distributing across the plate over time.

This is a visual representation of the plate before the simulation starts(initial condition):
<img src="https://github.com/openhackathons-org/gpubootcamp/raw/5c077a6c70989492f4bd850240fdcaf8b16fd555/hpc/openacc/English/Fortran/jupyter_notebook/images/plate1.png" width=600>



We can see that the plate is uniformly room temperature, except for the top edge. Within the laplace2d.f90 file, we see a function called `initialize`. This function is what **"heats"** the top edge of the plate.
```fortran
subroutine initialize(A, Anew, m, n)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(out)   :: A(:,:)
      real(fp_kind),allocatable,intent(out)   :: Anew(:,:)
      integer,intent(in)          :: m, n

      allocate ( A(0:n-1,0:m-1), Anew(0:n-1,0:m-1) )

      A    = 0.0_fp_kind
      Anew = 0.0_fp_kind

      A(0,:)    = 1.0_fp_kind
      Anew(0,:) = 1.0_fp_kind
    end subroutine initialize
```

After the top edge is heated, the code will simulate the heat distributing across the length of the plate. We will keep the top edge at a constant heat as the simulation progresses.



This is the plate after several iterations of our simulation:

<img src="https://github.com/openhackathons-org/gpubootcamp/raw/5c077a6c70989492f4bd850240fdcaf8b16fd555/hpc/openacc/English/Fortran/jupyter_notebook/images/plate2.png" width=600>

That's the theory: simple heat distribution. However, we are more interested in how the code works.

# Code Breakdown
The 2-dimensional plate is represented by a 2-dimensional array containing double-precision floating point values. These doubles represent temperature; 0.0 is room temperature, and 1.0 is our max temperature. The 2-dimensional plate has two states, one represents the current temperature, and one represents the expected temperature values at the next step in our simulation. These two states are represented by arrays **`A`** and **`Anew`** respectively. The following is a visual representation of these arrays, with the top edge "heated".

Simulating this state in two arrays is very important for our calcNext function. Our calcNext is essentially our "simulate" function. **`calcNext`** will look at the inner elements of A (meaning everything except for the edges of the plate) and update each elements temperature based on the temperature of its neighbors. If we attempted to calculate in-place (using only **`A`**), then each element would calculate its new temperature based on the updated temperature of previous elements. This data dependency not only prevents parallelizing the code, but would also result in incorrect results when run in serial. By calculating into the temporary array **`Anew`** we ensure that an entire step of our simulation has completed before updating the A array.


This is the **`calcNext`** function:

```fortran
01 function calcNext(A, Anew, m, n)
02   integer, parameter          :: fp_kind=kind(1.0d0)
03   real(fp_kind),intent(inout) :: A(0:n-1,0:m-1)
04   real(fp_kind),intent(inout) :: Anew(0:n-1,0:m-1)
05   integer,intent(in)          :: m, n
06   integer                     :: i, j
07   real(fp_kind)               :: error
08    
09   error=0.0_fp_kind
10    
11   do j=1,m-2
12     do i=1,n-2
13        Anew(i,j) = 0.25_fp_kind * ( A(i+1,j  ) + A(i-1,j  ) + &
14                                     A(i  ,j-1) + A(i  ,j+1) )
15        error = max( error, abs(Anew(i,j)-A(i,j)) )
16     end do
17   end do
18   calcNext = error
19 end function calcNext
```



We see on lines 13 and 14 where we are calculating the value of **`Anew`** at**` i,j`** by averaging the current values of its neighbors. Line 15 is where we calculate the current rate of change for the simulation by looking at how much the **`i,j`** element changed during this step and finding the maximum value for this **`error`**. This allows us to short-circuit our simulation if it reaches a steady state before we've completed our maximum number of iterations.

Lastly, our **`swap`** subroutine will copy the contents of **`Ane`**w to **`A`**.



```fortran
01 subroutine swap(A, Anew, m, n)
02   integer, parameter        :: fp_kind=kind(1.0d0)
03   real(fp_kind),intent(out) :: A(0:n-1,0:m-1)
04   real(fp_kind),intent(in)  :: Anew(0:n-1,0:m-1)
05   integer,intent(in)        :: m, n
06   integer                   :: i, j
07 
08   do j=1,m-2
09     do i=1,n-2
10       A(i,j) = Anew(i,j)
11     end do
12   end do
13 end subroutine swap
```



# Run the Code
Now that we've seen what the code does, let's build and run it. We need to record the results of our program before making any changes so that we can compare them to the results from the parallel code later on. It is also important to record the time that the program takes to run, as this will be our primary indicator to whether or not our parallelization is improving performance.

# Compiling the Code with NVIDIA HPC
For this lab we are using the NVIDIA HPC compiler to compiler our code. You will not need to memorize the compiler commands to complete this lab, however, they will be helpful to know if you want to parallelize your own personal code with OpenACC.

nvc : this is the command to compile C code
nvc++ : this is the command to compile C++ code
nvfortran : this is the command to compile Fortran code
-fast : this compiler flag instructs the compiler to use what it believes are the best possible optimizations for our system


```
FC := nvfortran
ACCFLAGS_1 := -fast

laplace_serial: laplace2d.f90 jacobi.f90 
	${FC} ${ACCFLAGS_1} -o laplace_serial laplace2d.f90 jacobi.f90


In [ ]:
!make clean && make laplace_serial && echo "Compilation Successful!" && ./laplace_serial

rm -f *.o laplace_serial laplace_multicore laplace_gpu
nvfortran -fast -o laplace_serial laplace2d.f90 jacobi.f90
laplace2d.f90:
jacobi.f90:
Compilation Successful!
Jacobi relaxation Calculation: 4096 x 4096 mesh
    0  0.250000
  100  0.002397
  200  0.001204
  300  0.000804
  400  0.000603
  500  0.000483
  600  0.000403
  700  0.000345
  800  0.000302
  900  0.000269
 completed in     50.510 seconds


# Expected Output



```
laplace2d.f90:
jacobi.f90:
Compilation Successful!
Jacobi relaxation Calculation: 4096 x 4096 mesh
    0  0.250000
  100  0.002397
  200  0.001204
  300  0.000804
  400  0.000603
  500  0.000483
  600  0.000403
  700  0.000345
  800  0.000302
  900  0.000269
 completed in     58.397 seconds
```



# Understanding Code Results
We see from the output that onces every hundred steps the program outputs the value of **`error`**, which is the maximum rate of change among the cells in our array. If these outputs change during any point while we parallelize our code, we know we've made a mistake. For simplicity, focus on the last output, which occurred at iteration 900 (the error value is 0.000269). It is also helpful to record the time the program took to run (it should have been about a minute). Our goal while parallelizing the code is ultimately to make it faster, so we need to know our "base runtime" in order to know if the code is running faster. Keep in mind that if you run the code multiple times you may get slightly different total runtimes, but you should get the same values for the error rates.

# Parallelizing Loops with OpenACC
At this point we know that most of the work done in our code happens in the **`calcNext`** and **`swap`** routines, so we'll focus our efforts there. We want the compiler to parallelize the loops in those two routines because that will give up the maximum speed-up over the baseline we just measured. To do this, we're going to use the OpenACC **` parallel loop`**  directive.



# OpenACC Directives
Using OpenACC directives will allow us to parallelize our code without needing to explicitly alter our code. What this means is that, by using OpenACC directives, we can have a single code that will function as both a sequential code and a parallel code.

## OpenACC Syntax
**`!$acc <directive> <clauses>`**

"**`!$acc`**" in Fortran is what's known as a "compiler hint" or "compiler directive." These are very similar to programmer comments, since the line begins with a comment statement "**`!`**". After the comment is "**`acc`**". OpenACC compliant compilers with appropriate command line options can interpret this as an OpenACC directive that "guide" the compiler above and beyond what the programming language allows. If the compiler does not understand "**`!$acc`**" it can ignore it, rather than throw a syntax error because it's just a comment.

**directives** are commands in OpenACC that will tell the compiler to do some action. For now, we will only use directives that allow the compiler to parallelize our code.

**clauses** are additions/alterations to our directives. These include (but are not limited to) optimizations. The way that I prefer to think about it: directives describe a general action for our compiler to do (such as, paralellize our code), and clauses allow the programmer to be more specific (such as, how we specifically want the code to be parallelized).



## Parallel and Loop Directives
There are three directives we will cover in this lab: **`parallel`**, **`loop`**, and **`parallel loop`**. Once we understand all three of them, you will be tasked with parallelizing our laplace code with your preferred directive (or use all of them, if you'd like!)

The parallel directive may be the most straight-forward of the directives. It will mark a region of the code for parallelization (this usually only includes parallelizing a single for loop.) Let's take a look:
```
!$acc parallel loop
    do i=1,N
        < loop code >
    enddo
```    
We may also define a parallel region. The parallel region may have multiple loops (though this is often not recommended!) The parallel region is everything contained within the outer-most curly braces.
```
!$acc parallel
    !$acc loop
    do i=1,N
        < loop code >
    enddo
!$acc end parallel    
```    

**`!$acc`** parallel loop will mark the next loop for parallelization. It is extremely important to include the loop, otherwise you will not be parallelizing the loop properly. The parallel directive tells the compiler to "redundantly parallelize" the code. The loop directive specifically tells the compiler that we want the loop parallelized. Let's look at an example of why the loop directive is so important. The **`parallel`** directive tells the compiler to create somewhere to run parallel code. OpenACC calls that somewhere a **`gang`**, which might be a thread on the CPU or maying a CUDA threadblock or OpenCL workgroup. It will choose how many gangs to create based on where you're running, only a few on a CPU (like 1 per CPU core) or lots on a GPU (1000's possibly). Gangs allow OpenACC code to scale from small CPUs to large GPUs because each one works completely independently of each other gang. That's why there's a space between gangs in the images below.



<img src="https://github.com/openhackathons-org/gpubootcamp/raw/5c077a6c70989492f4bd850240fdcaf8b16fd555/hpc/openacc/English/Fortran/jupyter_notebook/images/parallel1f.png" width=600>

<img src="https://github.com/openhackathons-org/gpubootcamp/raw/5c077a6c70989492f4bd850240fdcaf8b16fd555/hpc/openacc/English/Fortran/jupyter_notebook/images/parallel2f.png" width=600>

There's a good chance that I don't want my loop to be run redundantly in every gang though, that seems wasteful and potentially dangerous. Instead I want to instruct the compiler to break up the iterations of my loop and to run them in parallel on the gangs. To do that, I simply add a loop directive to the interesting loops. This instructs the compiler that I want my loop to be parallelized and promises to the compiler that it's safe to do so. Now that I have both parallel and loop, things loop a lot better (and run a lot faster). Now the compiler is spreading my loop iterations to all of my gangs, but also running multiple iterations of the loop at the same time within each gang as a vector. Think of a vector like this, I have 10 numbers that I want to add to 10 other numbers (in pairs). Rather than looking up each pair of numbers, adding them together, storing the result, and then moving on to the next pair in-order, modern computer hardware allows me to add all 10 pairs together all at once, which is a lot more efficient.

<img src="https://github.com/openhackathons-org/gpubootcamp/raw/5c077a6c70989492f4bd850240fdcaf8b16fd555/hpc/openacc/English/Fortran/jupyter_notebook/images/parallel3f.png" width=600>

The acc parallel loop directive is both a promise and a request to the compiler. I as the programmer am promising that the loop can safely be parallelized and am requesting that the compiler do so in a way that makes sense for the machine I am targeting. The compiler may make completely different decisions if I'm compiling for a multicore CPU than it would for a GPU and that's the idea. OpenACC enables programmers to parallelize their codes without having to worry about the details of how best to do so for every possible machine.-

## The Reduction Clause
There's one very important clause that you'll need to know for our example code: the **`reduction`** clause. Take note of how the loops in **`calcNext`** each calculate an error value and then compare against the maximum value to find an absolute maximum. When executing this operation in parallel, it's necessary to do a reduction in order to ensure you always get the correct answer. A reduction takes all of the values of **`error`** calculated in the loops and reduces them down to a single answer, in this case the maximum. There are a variety of reductions that can be done, but for our example code we only care about the max operation. We will inform the compiler about our reduction by adding a **`reduction(max:error)`** clause to the**` acc parallel loop `**in the **`calcNext`** function.

## Parallelize the Example Code
At this point you have all of the tools you need to begin accelerating your application. The loops you will be parallelizing are in laplace2d.f90.   

It is advisable to start with the **`calcNext`** routine and test your changes by compiling and running the code before moving on to the **`swap`** routine. OpenACC can be incrementally added to your application so that you can ensure each change is correct before getting too far along, which greatly simplifies debugging.

Once you have made your changes, you can compile and run the application by running the cell below. Please note that our compiler options have changed a little bit, we've added the following two important flags:

 - **`-ta=multicore `**- This instructs the compiler to build your OpenACC loops and to target them to run across the cores of a multicore CPU.

- **`-Minfo=accel`** - This instructs the compiler to give us some additional information about how it parallelized the code. We'll review how to interpret this feedback in just a moment.

Go ahead and build and run the code, noting both the error value at the 900th iteration and the total runtime. If the error value changed, you may have made a mistake. Don't forget the**` reduction(max:error)`** clause on the loop in the **`calcNext`** function!

In [ ]:
%%file laplace2d.f90

module laplace2d
  public :: initialize
  public :: calcNext
  public :: swap
  public :: dealloc
  contains
    subroutine initialize(A, Anew, m, n)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(out)   :: A(:,:)
      real(fp_kind),allocatable,intent(out)   :: Anew(:,:)
      integer,intent(in)          :: m, n

      allocate ( A(0:n-1,0:m-1), Anew(0:n-1,0:m-1) )

      A    = 0.0_fp_kind
      Anew = 0.0_fp_kind

      A(0,:)    = 1.0_fp_kind
      Anew(0,:) = 1.0_fp_kind
    end subroutine initialize

    function calcNext(A, Anew, m, n)
      integer, parameter          :: fp_kind=kind(1.0d0)
      real(fp_kind),intent(inout) :: A(0:n-1,0:m-1)
      real(fp_kind),intent(inout) :: Anew(0:n-1,0:m-1)
      integer,intent(in)          :: m, n
      integer                     :: i, j
      real(fp_kind)               :: error

      error=0.0_fp_kind

      ! ######### TODO  with reduction ################
      !$acc parallel
      !$acc loop

      do j=1,m-2
        do i=1,n-2
          Anew(i,j) = 0.25_fp_kind * ( A(i+1,j  ) + A(i-1,j  ) + &
                                       A(i  ,j-1) + A(i  ,j+1) )
          error = max( error, abs(Anew(i,j)-A(i,j)) )
        end do
      end do
    !$acc end parallel       
      calcNext = error
    end function calcNext

    subroutine swap(A, Anew, m, n)
      integer, parameter        :: fp_kind=kind(1.0d0)
      real(fp_kind),intent(out) :: A(0:n-1,0:m-1)
      real(fp_kind),intent(in)  :: Anew(0:n-1,0:m-1)
      integer,intent(in)        :: m, n
      integer                   :: i, j

      ! ######### TODO  ################
      !$acc parallel
      !$acc loop      
      do j=1,m-2
        do i=1,n-2
          A(i,j) = Anew(i,j)
        end do
      end do
!$acc end parallel       
    end subroutine swap

    subroutine dealloc(A, Anew)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(in) :: A
      real(fp_kind),allocatable,intent(in) :: Anew

      deallocate (A,Anew)
    end subroutine
end module laplace2d

Overwriting laplace2d.f90


```

FC := nvfortran

ACCFLAGS_2 := -fast -acc -ta=multicore -Minfo=accel

laplace_multicore: laplace2d.f90 jacobi.f90 
	${FC} ${ACCFLAGS_2} -o laplace_multicore laplace2d.f90 jacobi.f90 

```

In [ ]:
! make clean && make laplace_multicore && echo "Compilation Successful!" && ./laplace_multicore

rm -f *.o laplace_serial laplace_multicore laplace_gpu
nvfortran -fast -acc -ta=multicore -Minfo=accel -o laplace_multicore laplace2d.f90 jacobi.f90 
laplace2d.f90:
calcnext:
     34, Generating Multicore code
         37, !$acc loop gang
     37, Generating implicit reduction(max:error)
     38, Loop is parallelizable
swap:
     56, Generating Multicore code
         58, !$acc loop gang
     59, Loop is parallelizable
jacobi.f90:
Compilation Successful!
Jacobi relaxation Calculation: 4096 x 4096 mesh
    0  0.250000
  100  0.002397
  200  0.001204
  300  0.000804
  400  0.000603
  500  0.000483
  600  0.000403
  700  0.000345
  800  0.000302
  900  0.000269
 completed in     44.516 seconds


Here's the ouput you should see after running the above cell. Your total runtime may be slightly different, but it should be close. If you find yourself stuck on this part, you can take a look at our [solution](https://github.com/openhackathons-org/gpubootcamp/blob/5c077a6c70989492f4bd850240fdcaf8b16fd555/hpc/openacc/English/Fortran/source_code/lab1/solutions/laplace2d.parallel.f90).




```
laplace2d.f90:
calcnext:
     58, Generating Multicore code
         59, !$acc loop gang
     58, Generating reduction(max:error)
     60, Loop is parallelizable
swap:
     76, Generating Multicore code
         77, !$acc loop gang
     78, Loop is parallelizable
jacobi.f90:
Compilation Successful!
Jacobi relaxation Calculation: 4096 x 4096 mesh
    0  0.250000
  100  0.002397
  200  0.001204
  300  0.000804
  400  0.000603
  500  0.000483
  600  0.000403
  700  0.000345
  800  0.000302
  900  0.000269
 completed in     32.012 seconds
```

Great! Now our code is running nearly twice as fast by using all of the cores on our CPU, but I really want to run the code on a GPU. Once you have accelerated both loop nests in the example application and are sure you're getting correct results, you only need to change one compiler option to build the code for the GPUs on our node.

Here's the new compiler option we'll be using:

 - **`-ta=tesla:cc70,managed `**- Build the code for the NVIDIA Tesla GPU on our system, using managed memory
 
Notice above that I'm using something called managed memory for this task. Since our CPU and GPU each have their own physical memory I need to move the data between these memories. To simplify things this week, I'm telling the compiler to manage all of that data movement for me. Next week you'll learn how and why to manage the data movement yourself.

```
FC := nvfortran

ACCFLAGS_3 := -fast -acc -ta=tesla:managed -Minfo=accel

laplace_gpu: laplace2d.f90 jacobi.f90 
	${FC} ${ACCFLAGS_3} -o laplace_gpu laplace2d.f90 jacobi.f90 


In [ ]:
! make clean && make laplace_gpu && echo "Compilation Successful!" && ./laplace_gpu

rm -f *.o laplace_serial laplace_multicore laplace_gpu
nvfortran -fast -acc -ta=tesla:managed -Minfo=accel -o laplace_gpu laplace2d.f90 jacobi.f90 
laplace2d.f90:
calcnext:
     34, Generating NVIDIA GPU code
         37, !$acc loop gang ! blockidx%x
             Generating implicit reduction(max:error)
         38, !$acc loop vector(128) ! threadidx%x
     34, Generating implicit copyin(a(:n-1,:m-1)) [if not already present]
         Generating implicit copy(error) [if not already present]
         Generating implicit copyout(anew(1:n-2,1:m-2)) [if not already present]
     38, Loop is parallelizable
swap:
     56, Generating NVIDIA GPU code
         58, !$acc loop gang ! blockidx%x
         59, !$acc loop vector(128) ! threadidx%x
     56, Generating implicit copyout(a(1:n-2,1:m-2)) [if not already present]
         Generating implicit copyin(anew(1:n-2,1:m-2)) [if not already present]
     59, Loop is parallelizable
jacobi.f90:
Compilation Successful!
Jacobi relaxation Calculation: 

Wow! That ran a lot faster! This demonstrates the power of using OpenACC to accelerate an application. I made very minimal code changes and could run my code on multicore CPUs and GPUs by only changing my compiler option. Very cool!

Just for your reference, here's the timings I saw at each step of the process. Please keep in mind that your times may be just a little bit different

- original : 58s
- Multicore : 32 s
- GPU : 4 sec

So how did the compiler perform this miracle of speeding up my code on both the CPU and GPU? Let's compiler the compiler output from those two versions:

### CPU
```
calcnext:
     58, Generating Multicore code
         59, !$acc loop gang
     58, Generating reduction(max:error)
     60, Loop is parallelizable
```

### GPU
```
calcnext:
     58, Accelerator kernel generated
         Generating Tesla code
         58, Generating reduction(max:error)
         59, !$acc loop gang ! blockidx%x
         60, !$acc loop vector(128) ! threadidx%x
     58, Generating implicit copyin(a(:n-1,:m-1))
         Generating implicit copy(error)
         Generating implicit copyout(anew(1:n-2,1:m-2))
     60, Loop is parallelizable
```

Notice the differences on lines 59 and 60 . The compiler recognized that the loops could be parallelized, but chose to break up the work in different ways. In a future lab you will learn more about how OpenACC breaks up the work, but for now it's enough to know that the compiler understood the differences between these two processors and changed its plan to make sense for each.

# Conclusion
That's it, you now have the necessary tools to start using OpenACC in your application! In future labs you will learn about how to manage the CPU and GPU memories and how to optimize your loops to run faster, so be sure to attend each week of this online course and to do each lab.

# LAB2 
<img src="https://www.nvidia.com/content/dam/en-zz/Solutions/about-nvidia/logo-and-brand/02-nvidia-logo-color-wht-500x200-4c25-d@2x.png" height="100"></td></tr>
</table>

# prepare lab2 in colab

In [ ]:
!mkdir -p  /content/lab2

In [ ]:
cd /content/lab2

/content/lab2


In [ ]:
%%file Makefile 
# Copyright (c) 2020 NVIDIA Corporation.  All rights reserved. 
FC := nvfortran
ACCFLAGS_1 := -fast -ta=tesla -Minfo=accel
ACCFLAGS_2 := -fast -ta=tesla:managed -Minfo=accel

laplace_update: laplace2d.f90 jacobi.f90
	${FC} ${ACCFLAGS_1} -o laplace_update laplace2d.f90 jacobi.f90

laplace_no_update: laplace2d.f90 jacobi.f90
	${FC} ${ACCFLAGS_1} -o laplace_no_update laplace2d.f90 jacobi.f90

laplace_no_managed: laplace2d.f90 jacobi.f90
	${FC} ${ACCFLAGS_1} -o laplace laplace2d.f90 jacobi.f90

laplace_managed: laplace2d.f90 jacobi.f90
	${FC} ${ACCFLAGS_2} -o laplace_managed laplace2d.f90 jacobi.f90

clean:
	rm -f *.o laplace laplace_*

Overwriting Makefile


In [ ]:
%%file jacobi.f90 

program jacobi
  use laplace2d
  implicit none
  integer, parameter :: fp_kind=kind(1.0d0)
  integer, parameter :: n=4096, m=4096, iter_max=1000
  integer :: i, j, iter
  real(fp_kind), dimension (:,:), allocatable :: A, Anew
  real(fp_kind) :: tol=1.0e-6_fp_kind, error=1.0_fp_kind
  real(fp_kind) :: start_time, stop_time

  call initialize(A, Anew, m, n)
   
  write(*,'(a,i5,a,i5,a)') 'Jacobi relaxation Calculation:', n, ' x', m, ' mesh'
 
  call cpu_time(start_time) 

  iter=0
  
  do while ( error .gt. tol .and. iter .lt. iter_max )

    error = calcNext(A, Anew, m, n)
    call swap(A, Anew, m, n)

    if(mod(iter,100).eq.0 ) write(*,'(i5,f10.6)'), iter, error

    iter = iter + 1

  end do

  call cpu_time(stop_time) 
  write(*,'(a,f10.3,a)')  ' completed in ', stop_time-start_time, ' seconds'

  deallocate (A,Anew)
end program jacobi

Overwriting jacobi.f90


In [ ]:
%%file laplace2d.f90 

module laplace2d
  public :: initialize
  public :: calcNext
  public :: swap
  public :: dealloc
  contains
    subroutine initialize(A, Anew, m, n)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(out)   :: A(:,:)
      real(fp_kind),allocatable,intent(out)   :: Anew(:,:)
      integer,intent(in)          :: m, n

      allocate ( A(0:n-1,0:m-1), Anew(0:n-1,0:m-1) )

      A    = 0.0_fp_kind
      Anew = 0.0_fp_kind

      A(0,:)    = 1.0_fp_kind
      Anew(0,:) = 1.0_fp_kind
    end subroutine initialize

    function calcNext(A, Anew, m, n)
      integer, parameter          :: fp_kind=kind(1.0d0)
      real(fp_kind),intent(inout) :: A(0:n-1,0:m-1)
      real(fp_kind),intent(inout) :: Anew(0:n-1,0:m-1)
      integer,intent(in)          :: m, n
      integer                     :: i, j
      real(fp_kind)               :: error

      error=0.0_fp_kind

      !$acc parallel loop reduction(max:error)
      do j=1,m-2
        do i=1,n-2
          Anew(i,j) = 0.25_fp_kind * ( A(i+1,j  ) + A(i-1,j  ) + &
                                       A(i  ,j-1) + A(i  ,j+1) )
          error = max( error, abs(Anew(i,j)-A(i,j)) )
        end do
      end do
      calcNext = error
    end function calcNext

    subroutine swap(A, Anew, m, n)
      integer, parameter        :: fp_kind=kind(1.0d0)
      real(fp_kind),intent(out) :: A(0:n-1,0:m-1)
      real(fp_kind),intent(in)  :: Anew(0:n-1,0:m-1)
      integer,intent(in)        :: m, n
      integer                   :: i, j

      !$acc parallel loop
      do j=1,m-2
        do i=1,n-2
          A(i,j) = Anew(i,j)
        end do
      end do
    end subroutine swap

    subroutine dealloc(A, Anew)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(in) :: A
      real(fp_kind),allocatable,intent(in) :: Anew

      deallocate (A,Anew)
    end subroutine
end module laplace2d

Overwriting laplace2d.f90



## Run the Code (With Managed Memory)
In the previous lab, we added OpenACC loop directives and relied on a feature called CUDA Managed Memory to deal with the separate CPU & GPU memories for us. Just adding OpenACC to our two loop nests we achieved a considerable performance boost. However, managed memory is not compatible with all GPUs or all compilers and it sometimes performs worse than programmer-defined memory management. Let's start with our solution from the previous lab and use this as our performance baseline. Note the runtime from the follow cell.

In [ ]:
! make clean && make laplace_managed && ./laplace_managed

rm -f *.o laplace laplace_*
nvfortran -fast -ta=tesla:managed -Minfo=accel -o laplace_managed laplace2d.f90 jacobi.f90
laplace2d.f90:
calcnext:
     33, Generating NVIDIA GPU code
         34, !$acc loop gang ! blockidx%x
             Generating reduction(max:error)
         35, !$acc loop vector(128) ! threadidx%x
     33, Generating implicit copyin(a(:n-1,:m-1)) [if not already present]
         Generating implicit copy(error) [if not already present]
         Generating implicit copyout(anew(1:n-2,1:m-2)) [if not already present]
     35, Loop is parallelizable
swap:
     51, Generating NVIDIA GPU code
         52, !$acc loop gang ! blockidx%x
         53, !$acc loop vector(128) ! threadidx%x
     51, Generating implicit copyout(a(1:n-2,1:m-2)) [if not already present]
         Generating implicit copyin(anew(1:n-2,1:m-2)) [if not already present]
     53, Loop is parallelizable
jacobi.f90:
Jacobi relaxation Calculation: 4096 x 4096 mesh
    0  0.250000
  100  0.002397
  200  0.0012

## Building Without Managed Memory
For this exercise we ultimately don't want to use CUDA Managed Memory, so let's removed the managed option from our compiler options. Try building and running the code now. What happens?

In [ ]:
! make clean && make laplace_no_managed && ./laplace

rm -f *.o laplace laplace_*
nvfortran -fast -ta=tesla -Minfo=accel -o laplace laplace2d.f90 jacobi.f90
laplace2d.f90:
calcnext:
     33, Generating NVIDIA GPU code
         34, !$acc loop gang ! blockidx%x
             Generating reduction(max:error)
         35, !$acc loop vector(128) ! threadidx%x
     33, Generating implicit copyin(a(:n-1,:m-1)) [if not already present]
         Generating implicit copy(error) [if not already present]
         Generating implicit copyout(anew(1:n-2,1:m-2)) [if not already present]
     35, Loop is parallelizable
swap:
     51, Generating NVIDIA GPU code
         52, !$acc loop gang ! blockidx%x
         53, !$acc loop vector(128) ! threadidx%x
     51, Generating implicit copyout(a(1:n-2,1:m-2)) [if not already present]
         Generating implicit copyin(anew(1:n-2,1:m-2)) [if not already present]
     53, Loop is parallelizable
jacobi.f90:
Jacobi relaxation Calculation: 4096 x 4096 mesh
    0  0.250000
  100  0.002397
  200  0.001204
  300  0.0008

OK, so we're able to run, but we're running very slowly. We'll address that in just a moment, but first we should address an issue that you might encounter if you ever try this same exercise in C/C++. Because Fortran arrays contain all of the necessary size and shape information for the compiler to move them to and from the device, this step works. Had you been programming in C/C++, however, the compiler would lack the information to move the arrays. Instead, you would have seen the following:

```
jacobi.c:
laplace2d.c:
NVC++-S-0155-Compiler failed to translate accelerator region (see -Minfo messages): Could not find allocated-variable index for symbol (laplace2d.c: 47)
calcNext:
     47, Accelerator kernel generated
         Generating Tesla code
         48, #pragma acc loop gang /* blockIdx.x */
         50, #pragma acc loop vector(128) /* threadIdx.x */
         54, Generating implicit reduction(max:error)
     48, Accelerator restriction: size of the GPU copy of Anew,A is unknown
     50, Loop is parallelizable
NVC++-F-0704-Compilation aborted due to previous errors. (laplace2d.c)
NVC++/x86-64 Linux 18.7-0: compilation aborted
This error message is not very intuitive, so let me explain it to you.:
```

- NVC++-S-0155-Compiler failed to translate accelerator region (see -Minfo messages): Could not find allocated-variable index for symbol (laplace2d.c: 47) - The compiler doesn't like something about a variable from line 47 of our code.

- 48, Accelerator restriction: size of the GPU copy of Anew,A is unknown - I don't see any further information about line 47, but at line 48 the compiler is struggling to understand the size and shape of the arrays Anew and A. It turns out, this is our problem.

So, what these cryptic compiler errors are telling us is that the compiler needs to create copies of A and Anew on the GPU in order to run our code there, but it doesn't know how big they are, so it's giving up. We would need to give the compiler more information about these arrays before it can move forward in C/C++. In Fortran the compiler can help us out, but the goal here is to explicitly manage your data, so let's find out how to do that.

OpenACC Data Clauses
Data clauses allow the programmer to specify data transfers between the host and device (or in our case, the CPU and the GPU). Because they are clauses, they can be added to other directives, such as the parallel loop directive that we used in the previous lab. Let's look at an example where we do not use a data clause.
```
allocate(A(N))

  !$acc parallel loop
  do i=1,100
    A(i) = 0
  enddo
```  
We have allocated an array A outside of our parallel region. This means that A is allocated in the CPU memory. However, we access A inside of our loop, and that loop is contained within a parallel region. Within that parallel region, A(i) is attempting to access a memory location within the GPU memory. We didn't explicitly allocate A on the GPU, so one of two things will happen.

The compiler will understand what we are trying to do, and automatically copy A from the CPU to the GPU.
The program will check for an array A in GPU memory, it won't find it, and it will throw an error.
Instead of hoping that we have a compiler that can figure this out, we could instead use a data clause.

```
allocate(A(N))

  !$acc parallel loop copy(A(1:N))
  do i=1,100
    A(i) = 0
  enddo
```  
We will learn the copy data clause first, because it is the easiest to use. With the inclusion of the copy data clause, our program will now copy the content of A from the CPU memory, into GPU memory. Then, during the execution of the loop, it will properly access A from the GPU memory. After the parallel region is finished, our program will copy A from the GPU memory back to the CPU memory. Let's look at one more direct example.

```
allocate(A(N))

  do i=1,100
    A(i) = 0
  enddo

  !$acc parallel loop copy(A(1:N))
  do i=1,100
    A(i) = 1
  enddo
```  
Now we have two loops; the first loop will execute on the CPU (since it does not have an OpenACC parallel directive), and the second loop will execute on the GPU. Array A will be allocated on the CPU, and then the first loop will execute. This loop will set the contents of A to be all 0. Then the second loop is encountered; the program will copy the array A (which is full of 0's) into GPU memory. Then, we will execute the second loop on the GPU. This will edit the GPU's copy of A to be full of 1's.

At this point, we have two seperate copies of A. The CPU copy is full of 0's, and the GPU copy is full of 1's. Now, after the parallel region finishes, the program will copy A back from the GPU to the CPU. After this copy, both the CPU and the GPU will contain a copy of A that contains all 1's. The GPU copy of A will then be deallocated.

This image offers another step-by-step example of using the copy clause.



<img src="https://github.com/openhackathons-org/gpubootcamp/raw/5c077a6c70989492f4bd850240fdcaf8b16fd555/hpc/openacc/English/Fortran/jupyter_notebook/images/copy_step_by_step.png" width=800>

We are also able to copy multiple arrays at once by using the following syntax.
```
!$acc parallel loop copy(A(1:N), B(1:N))
do i = 1, N
    A(i) = B(i)
end do
```
Of course, we might not want to copy our data both to and from the GPU memory. Maybe we only need the array's values as inputs to the GPU region, or maybe it's only the final results we care about, or perhaps the array is only used temporarily on the GPU and we don't want to copy it either directive. The following OpenACC data clauses provide a bit more control than just the copy clause.

 - **`copyin`** - Create space for the array and copy the input values of the array to the device. At the end of the region, the array is deleted without copying anything back to the host.
 - **`copyout`** - Create space for the array on the device, but don't initialize it to anything. At the end of the region, copy the results back and then delete the device array.
 - **`create`** - Create space of the array on the device, but do not copy anything to the device at the beginning of the region, nor back to the host at the end. The array will be deleted from the device at the end of the region.
 - **`present`** - Don't do anything with these variables. I've put them on the device somewhere else, so just assume they're available.

You may also use them to operate on multiple arrays at once, by including those arrays as a comma separated list.

`!$acc parallel loop copy( A(1:N), B(1:M), C(1:Q) )`
You may also use more than one data clause at a time.

`!$acc parallel loop create( A(1:N) ) copyin( B(1:M) ) copyout( C(1:Q) )`


## Array Shaping

The shape of the array specifies how much data needs to be transferred. Let's look at an example:



```fortran
!$acc parallel loop copy(A(1:N))
do i = 1, N
  A(i) = 0
end do
```


Focusing specifically on the copy(A(1:N)), the shape of the array is defined within the brackets. The syntax for array shape is (starting_index:ending_index). This means that (in the code example) we are copying data from array A, starting at index 1 (the start of the array), and copying until index N (which is most likely the length of the entire array).

We are also able to only copy a portion of the array:

`!$acc parallel loop copy(A(2:N-2))`
This would copy all of the elements of A except for the first, and last element.

Lastly, if you do not specify a starting index, 1 is assumed. This means that

`!$acc parallel loop copy(A(1:N))`
is equivalent to

`!$acc parallel loop copy(A(:N))`
And since we're in Fortran, this can be shorted even more to just

`!$acc parallel loop copy(A(:))`

## Making the Sample Code Work with Explicit Data Management
In order to build our example code without CUDA managed memory but with explicit programmer-controlled data management, we need to give the compiler more information about the arrays. How do our two loop nests use the arrays A and Anew? The calcNext function take A as input and generates Anew as output, but also needs Anew copied in because we need to maintain that hot boundary at the top. So you will want to add a copyin clause for A and a copy clause for Anew on your region. The swap function takes Anew as input and A as output, so it needs the exact opposite data clauses. It's also necessary to tell the compiler the size of the two arrays by using array shaping. Our arrays are m times n in size, so we'll tell the compiler their shape starts at 0 and has n*m elements, using the syntax above. Go ahead and add data clauses to the two parallel loop directives in laplace2d.f90.

modify**` laplace2d.f90`** Then try to build again.

In [ ]:
%%file laplace2d.f90

module laplace2d
  public :: initialize
  public :: calcNext
  public :: swap
  public :: dealloc
  contains
    subroutine initialize(A, Anew, m, n)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(out)   :: A(:,:)
      real(fp_kind),allocatable,intent(out)   :: Anew(:,:)
      integer,intent(in)          :: m, n

      allocate ( A(0:n-1,0:m-1), Anew(0:n-1,0:m-1) )

      A    = 0.0_fp_kind
      Anew = 0.0_fp_kind

      A(0,:)    = 1.0_fp_kind
      Anew(0,:) = 1.0_fp_kind
    end subroutine initialize

    function calcNext(A, Anew, m, n)
      integer, parameter          :: fp_kind=kind(1.0d0)
      real(fp_kind),intent(inout) :: A(0:n-1,0:m-1)
      real(fp_kind),intent(inout) :: Anew(0:n-1,0:m-1)
      integer,intent(in)          :: m, n
      integer                     :: i, j
      real(fp_kind)               :: error

      error=0.0_fp_kind

      !$acc parallel loop reduction(max:error) copyin(A(:,:)) copy(Anew(:,:))
      do j=1,m-2
        do i=1,n-2
          Anew(i,j) = 0.25_fp_kind * ( A(i+1,j  ) + A(i-1,j  ) + &
                                       A(i  ,j-1) + A(i  ,j+1) )
          error = max( error, abs(Anew(i,j)-A(i,j)) )
        end do
      end do
      calcNext = error
    end function calcNext

    subroutine swap(A, Anew, m, n)
      integer, parameter        :: fp_kind=kind(1.0d0)
      real(fp_kind),intent(out) :: A(0:n-1,0:m-1)
      real(fp_kind),intent(in)  :: Anew(0:n-1,0:m-1)
      integer,intent(in)        :: m, n
      integer                   :: i, j

      !$acc parallel loop copyin(Anew(:,:)) copyout(A(:,:))
      do j=1,m-2
        do i=1,n-2
          A(i,j) = Anew(i,j)
        end do
      end do
    end subroutine swap

    subroutine dealloc(A, Anew)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(in) :: A
      real(fp_kind),allocatable,intent(in) :: Anew

      deallocate (A,Anew)
    end subroutine
end module laplace2d

Overwriting laplace2d.f90


In [ ]:
!make clean && make laplace_no_managed && ./laplace

rm -f *.o laplace laplace_*
nvfortran -fast -ta=tesla -Minfo=accel -o laplace laplace2d.f90 jacobi.f90
laplace2d.f90:
calcnext:
     33, Generating copyin(a(:,:)) [if not already present]
         Generating NVIDIA GPU code
         34, !$acc loop gang ! blockidx%x
             Generating reduction(max:error)
         35, !$acc loop vector(128) ! threadidx%x
     33, Generating implicit copy(error) [if not already present]
         Generating copy(anew(:,:)) [if not already present]
     35, Loop is parallelizable
swap:
     51, Generating copyout(a(:,:)) [if not already present]
         Generating copyin(anew(:,:)) [if not already present]
         Generating NVIDIA GPU code
         52, !$acc loop gang ! blockidx%x
         53, !$acc loop vector(128) ! threadidx%x
     53, Loop is parallelizable
jacobi.f90:
Jacobi relaxation Calculation: 4096 x 4096 mesh
    0  0.250000
  100  0.002397
  200  0.001204
  300  0.000804
  400  0.000603
  500  0.000483
  600  0.000403
  700  0.000345
  

Hint : [solution](https://github.com/openhackathons-org/gpubootcamp/blob/5c077a6c70989492f4bd850240fdcaf8b16fd555/hpc/openacc/English/Fortran/source_code/lab2/solutions/laplace2d.f90)

Well, the good news is that it should have built correctly and run. If it didn't, check your data clauses carefully. The bad news is that now it runs a whole lot slower than it did in the last exercise. Let's try to figure out why. The NVIDIA HPC compiler provides your executable with built-in timers, so let's start by enabling them and seeing what it shows. You can enable these timers by setting the environment variable **`PGI_ACC_TIME=1`**. Run the cell below to get the program output with the built-in profiler enabled.

In [ ]:
!make clean && make laplace_no_managed && PGI_ACC_TIME=1 ./laplace 

rm -f *.o laplace laplace_*
nvfortran -fast -ta=tesla -Minfo=accel -o laplace laplace2d.f90 jacobi.f90
laplace2d.f90:
calcnext:
     33, Generating copyin(a(:,:)) [if not already present]
         Generating NVIDIA GPU code
         34, !$acc loop gang ! blockidx%x
             Generating reduction(max:error)
         35, !$acc loop vector(128) ! threadidx%x
     33, Generating implicit copy(error) [if not already present]
         Generating copy(anew(:,:)) [if not already present]
     35, Loop is parallelizable
swap:
     51, Generating copyout(a(:,:)) [if not already present]
         Generating copyin(anew(:,:)) [if not already present]
         Generating NVIDIA GPU code
         52, !$acc loop gang ! blockidx%x
         53, !$acc loop vector(128) ! threadidx%x
     53, Loop is parallelizable
jacobi.f90:
Jacobi relaxation Calculation: 4096 x 4096 mesh
libcupti.so not found
    0  0.250000
  100  0.002397
  200  0.001204
  300  0.000804
  400  0.000603
  500  0.000483
  600  0.000

Your output should look something like what you see below.
```
 completed in    182.941 seconds

Accelerator Kernel Timing data
/content/lab2/laplace2d.f90
  calcnext  NVIDIA  devicenum=0
    time(us): 53,265,706
    58: compute region reached 1000 times
        58: kernel launched 1000 times
            grid: [4094]  block: [128]
             device time(us): total=2,398,959 max=2,415 min=2,380 avg=2,398
            elapsed time(us): total=2,454,466 max=2,489 min=2,435 avg=2,454
        58: reduction kernel launched 1000 times
            grid: [2]  block: [256]
             device time(us): total=19,011 max=20 min=19 avg=19
            elapsed time(us): total=46,816 max=67 min=43 avg=46
    58: data region reached 4000 times
        58: data copyin transfers: 17000
             device time(us): total=33,881,820 max=2,141 min=6 avg=1,993
        66: data copyout transfers: 10000
             device time(us): total=16,965,916 max=2,135 min=9 avg=1,696
/content/lab2/laplace2d.f90
  swap  NVIDIA  devicenum=0
    time(us): 36,205,726
    76: compute region reached 1000 times
        76: kernel launched 1000 times
            grid: [4094]  block: [128]
             device time(us): total=2,306,221 max=2,319 min=2,293 avg=2,306
            elapsed time(us): total=2,363,356 max=2,397 min=2,347 avg=2,363
    76: data region reached 2000 times
        76: data copyin transfers: 8000
             device time(us): total=16,942,581 max=2,141 min=2,114 avg=2,117
        82: data copyout transfers: 9000
             device time(us): total=16,956,924 max=2,134 min=13 avg=1,884

```        

The total runtime was roughly 130 seconds with the profiler turned on (roughly 120 without). We can see that calcNext required roughly 53 seconds to run by looking at the time(us) line under the calcNext line. We can also look at the data region section and determine that 34 seconds were spent copying data to the device and 17 seconds copying data out for the device. The swap function has very similar numbers. That means that the program is actually spending very little of its runtime doing calculations. Why is the program copying so much data around? The screenshot below comes from the Nsight Systems profiler and shows part of one step of our outer while loop. The greenish and pink colors are data movement and the blue colors are our kernels (calcNext and swap). Notice that for each kernel we have copies to the device (greenish) before and copies from the device (pink) after. The means we have 4 segments of data copies for every iteration of the outer while loop.

Let's contrast this with the managed memory version. The image below shows the same program built with managed memory. Notice that there's a lot of "data migration" at the beginning, where the data is first used, but there's no data movement between the loops. This tells me that the data movement isn't really needed between these loops, but we need to tell the compiler that.

Because the loops are in two separate files, the compiler can't really see that the data is reused on the GPU between those function. We need to move our data movement up to a higher level where we can reuse it for each step through the program. To do that, we'll add OpenACC data directives.



---


# OpenACC Structured Data Directive
The OpenACC data directives allow the programmer to explicitly manage the data on the device (in our case, the GPU). Specifically, the structured data directive will mark a static region of our code as a data region.
```
< Initialize data on host (CPU) >
 
!$acc data < data clauses >

    < Code >

!$acc end data
```
Device memory allocation happens at the beginning of the region, and device memory deallocation happens at the end of the region. Additionally, any data movement from the host to the device (CPU to GPU) happens at the beginning of the region, and any data movement from the device to the host (GPU to CPU) happens at the end of the region. Memory allocation/deallocation and data movement is defined by which clauses the programmer includes. This is a list of the most important data clauses that we can use:

Encompassing Multiple Compute Regions
A single data region can contain any number of parallel/kernels regions. Take the following example:
```
!$acc data copyin(A(1:N), B(1:N)) create(C(1:N))

    !$acc parallel loop
    do i = 1, N
        C(i) = A(i) + B(i)
    end do

    !$acc parallel loop
    do i = 1, N
        A(i) = C(i) + B(i)
    end do

!$acc end data
```
You may also encompass function calls within the data region:
```
subroutine copy(A, B, N)

    !$acc parallel loop
    do i = 1, N
        A(i) = B(i)
    end do

end subroutine

...

!$acc data copyout(A(1:N),B(1:N)) copyin(C(1:N))

    call copy(A, C, N)

    call copy(A, B, N)
!$acc end data
```


> Indented block



## Adding the Structured Data Directive to our Code
Add a structured data directive to properly handle the arrays A and Anew. We've already added data clauses to our two functions, so this time we'll move up the calltree and add a structured data region around our while loop in the main program. Think about the input and output to this while loop and choose your data clauses for A and Anew accordingly.
modify **`jacobi.f90`** Then, run the following script to check you solution. You code should run just as good as (or slightly better) than our managed memory code.

In [ ]:
%%file jacobi.f90


program jacobi
  use laplace2d
  implicit none
  integer, parameter :: fp_kind=kind(1.0d0)
  integer, parameter :: n=4096, m=4096, iter_max=1000
  integer :: i, j, iter
  real(fp_kind), dimension (:,:), allocatable :: A, Anew
  real(fp_kind) :: tol=1.0e-6_fp_kind, error=1.0_fp_kind
  real(fp_kind) :: start_time, stop_time

  call initialize(A, Anew, m, n)
   
  write(*,'(a,i5,a,i5,a)') 'Jacobi relaxation Calculation:', n, ' x', m, ' mesh'
 
  call cpu_time(start_time) 

  iter=0
  
  !$acc data copy(A(:,:)) create(Anew(:,:))
  do while ( error .gt. tol .and. iter .lt. iter_max )

    error = calcNext(A, Anew, m, n)
    call swap(A, Anew, m, n)

    if(mod(iter,100).eq.0 ) write(*,'(i5,f10.6)'), iter, error

    iter = iter + 1

  end do
  !$acc end data

  call cpu_time(stop_time) 
  write(*,'(a,f10.3,a)')  ' completed in ', stop_time-start_time, ' seconds'

  deallocate (A,Anew)
end program jacobi

Overwriting jacobi.f90


In [ ]:
 ! make clean && make laplace_no_managed && ./laplace

rm -f *.o laplace laplace_*
nvfortran -fast -ta=tesla -Minfo=accel -o laplace laplace2d.f90 jacobi.f90
laplace2d.f90:
calcnext:
     33, Generating copyin(a(:,:)) [if not already present]
         Generating NVIDIA GPU code
         34, !$acc loop gang ! blockidx%x
             Generating reduction(max:error)
         35, !$acc loop vector(128) ! threadidx%x
     33, Generating implicit copy(error) [if not already present]
         Generating copy(anew(:,:)) [if not already present]
     35, Loop is parallelizable
swap:
     51, Generating copyout(a(:,:)) [if not already present]
         Generating copyin(anew(:,:)) [if not already present]
         Generating NVIDIA GPU code
         52, !$acc loop gang ! blockidx%x
         53, !$acc loop vector(128) ! threadidx%x
     53, Loop is parallelizable
jacobi.f90:
jacobi:
     21, Generating create(anew(anew$sd1:(anew$sd1-1)+anew$sd1,anew$sd1:(anew$sd1-1)+anew$sd1)) [if not already present]
         Generating copy(a(a$sd2:(a$sd2-1)+a$sd2,

In [ ]:
!make clean && make laplace_no_managed && PGI_ACC_TIME=1 ./laplace 

rm -f *.o laplace laplace_*
nvfortran -fast -ta=tesla -Minfo=accel -o laplace laplace2d.f90 jacobi.f90
laplace2d.f90:
calcnext:
     33, Generating copyin(a(:,:)) [if not already present]
         Generating NVIDIA GPU code
         34, !$acc loop gang ! blockidx%x
             Generating reduction(max:error)
         35, !$acc loop vector(128) ! threadidx%x
     33, Generating implicit copy(error) [if not already present]
         Generating copy(anew(:,:)) [if not already present]
     35, Loop is parallelizable
swap:
     51, Generating copyout(a(:,:)) [if not already present]
         Generating copyin(anew(:,:)) [if not already present]
         Generating NVIDIA GPU code
         52, !$acc loop gang ! blockidx%x
         53, !$acc loop vector(128) ! threadidx%x
     53, Loop is parallelizable
jacobi.f90:
jacobi:
     21, Generating create(anew(anew$sd1:(anew$sd1-1)+anew$sd1,anew$sd1:(anew$sd1-1)+anew$sd1)) [if not already present]
         Generating copy(a(a$sd2:(a$sd2-1)+a$sd2,

# OpenACC Update Directive

When we use the data clauses you are only able to copy data between host and device memory at the beginning and end of your regions, but what if you need to copy data in the middle? For example, what if we wanted to debug our code by printing out the array every 100 steps to make sure it looks right. In order to transfer data at those times, we can use the **`update`** directive. The update directive will explicitly transfer data between the host and the device. The **`update`** directive has two clauses:

- **`self`** - The self clause will transfer data from the device to the host (GPU to CPU). You will sometimes see this clause called the **`host`** clause.
- **`device`** - The device clause will transfer data from the host to the device (CPU to GPU).
The syntax would look like:

**`!$acc update self(A(1:N))`**
 
**`!$acc update device(A(1:N))`**

All of the array shaping rules apply.

As an example, let's create a version of our laplace code where we want to print the array A after every 100 iterations of our loop. The code will look like this:
```
!acc data copyin( A(n,m), Anew(n,m))

do while (error > tol && iter < iter_max)
    error = calcNext(A, Anew, m, n)
    swap(A, Anew, m, n)

    if(mod(iter,100).eq.0 ) then
      write(*,'(i5,f10.6)'), iter, error
      do i=1,n
        do j=1,m
          write(*,'(f10.2)', advance="no"), A(i,j)
        enddo
      enddo
    end if

    iter = iter+1

end do

!$acc end data
```
Let's run this code (on a very small data set, so that we don't overload the console by printing thousands of numbers; we've set m = n = 10).

In [ ]:
%%file jacobi.f90


program jacobi
  use laplace2d
  implicit none
  integer, parameter :: fp_kind=kind(1.0d0)
  integer, parameter :: n=10, m=10, iter_max=1000
  integer :: i, j, iter
  real(fp_kind), dimension (:,:), allocatable :: A, Anew
  real(fp_kind) :: tol=1.0e-6_fp_kind, error=1.0_fp_kind
  real(fp_kind) :: start_time, stop_time

  call initialize(A, Anew, m, n)
   
  write(*,'(a,i5,a,i5,a)') 'Jacobi relaxation Calculation:', n, ' x', m, ' mesh'
 
  call cpu_time(start_time) 

  iter=0
  
  !$acc data copyin(A, Anew)
  do while ( error .gt. tol .and. iter .lt. iter_max )

    error = calcNext(A, Anew, m, n)
    call swap(A, Anew, m, n)

    if(mod(iter,100).eq.0 ) then
      write(*,'(i5,f10.6)'), iter, error
      do i=1,n 
        do j=1,m 
          write(*,'(f10.2)', advance="no"), A(i,j) 
        enddo 
      enddo
    end if

    iter = iter + 1

  end do
  !$acc end data

  call cpu_time(stop_time) 
  write(*,'(a,f10.3,a)')  ' completed in ', stop_time-start_time, ' seconds'

  deallocate (A,Anew)
end program jacobi

Overwriting jacobi.f90


In [ ]:
%%file laplace2d.f90

module laplace2d
  public :: initialize
  public :: calcNext
  public :: swap
  public :: dealloc
  contains
    subroutine initialize(A, Anew, m, n)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(out)   :: A(:,:)
      real(fp_kind),allocatable,intent(out)   :: Anew(:,:)
      integer,intent(in)          :: m, n

      allocate ( A(0:n-1,0:m-1), Anew(0:n-1,0:m-1) )

      A    = 0.0_fp_kind
      Anew = 0.0_fp_kind

      A(0,:)    = 1.0_fp_kind
      Anew(0,:) = 1.0_fp_kind
    end subroutine initialize

    function calcNext(A, Anew, m, n)
      integer, parameter          :: fp_kind=kind(1.0d0)
      real(fp_kind),intent(inout) :: A(0:n-1,0:m-1)
      real(fp_kind),intent(inout) :: Anew(0:n-1,0:m-1)
      integer,intent(in)          :: m, n
      integer                     :: i, j
      real(fp_kind)               :: error

      error=0.0_fp_kind

      !$acc parallel loop reduction(max:error)
      do j=1,m-2
        do i=1,n-2
          Anew(i,j) = 0.25_fp_kind * ( A(i+1,j  ) + A(i-1,j  ) + &
                                       A(i  ,j-1) + A(i  ,j+1) )
          error = max( error, abs(Anew(i,j)-A(i,j)) )
        end do
      end do
      calcNext = error
    end function calcNext

    subroutine swap(A, Anew, m, n)
      integer, parameter        :: fp_kind=kind(1.0d0)
      real(fp_kind),intent(out) :: A(0:n-1,0:m-1)
      real(fp_kind),intent(in)  :: Anew(0:n-1,0:m-1)
      integer,intent(in)        :: m, n
      integer                   :: i, j

      !$acc parallel loop
      do j=1,m-2
        do i=1,n-2
          A(i,j) = Anew(i,j)
        end do
      end do
    end subroutine swap

    subroutine dealloc(A, Anew)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(in) :: A
      real(fp_kind),allocatable,intent(in) :: Anew

      deallocate (A,Anew)
    end subroutine
end module laplace2d

Overwriting laplace2d.f90


In [ ]:
!make clean && make laplace_no_update && ./laplace_no_update 10 10

rm -f *.o laplace laplace_*
nvfortran -fast -ta=tesla -Minfo=accel -o laplace_no_update laplace2d.f90 jacobi.f90
laplace2d.f90:
calcnext:
     34, Generating NVIDIA GPU code
         35, !$acc loop gang ! blockidx%x
             Generating reduction(max:error)
         36, !$acc loop vector(128) ! threadidx%x
     34, Generating implicit copyin(a(:n-1,:m-1)) [if not already present]
         Generating implicit copy(error) [if not already present]
         Generating implicit copyout(anew(1:n-2,1:m-2)) [if not already present]
     36, Loop is parallelizable
swap:
     52, Generating NVIDIA GPU code
         53, !$acc loop gang ! blockidx%x
         54, !$acc loop vector(128) ! threadidx%x
     52, Generating implicit copyout(a(1:n-2,1:m-2)) [if not already present]
         Generating implicit copyin(anew(1:n-2,1:m-2)) [if not already present]
     54, Loop is parallelizable
jacobi.f90:
jacobi:
     21, Generating copyin(anew(:,:),a(:,:)) [if not already present]
Jacobi relaxation Cal

In [ ]:
%%file jacobi.f90


program jacobi
  use laplace2d
  implicit none
  integer, parameter :: fp_kind=kind(1.0d0)
  integer, parameter :: n=10, m=10, iter_max=1000
  integer :: i, j, iter
  real(fp_kind), dimension (:,:), allocatable :: A, Anew
  real(fp_kind) :: tol=1.0e-6_fp_kind, error=1.0_fp_kind
  real(fp_kind) :: start_time, stop_time

  allocate ( A(0:n-1,0:m-1), Anew(0:n-1,0:m-1) )

  call initialize(A, Anew, m, n)
   
  write(*,'(a,i5,a,i5,a)') 'Jacobi relaxation Calculation:', n, ' x', m, ' mesh'
 
  call cpu_time(start_time) 

  iter=0
  
  !$acc data copyin(A, Anew)
  do while ( error .gt. tol .and. iter .lt. iter_max )

    error = calcNext(A, Anew, m, n)
    call swap(A, Anew, m, n)

    if(mod(iter,100).eq.0 ) then
      write(*,'(i5,f10.6)'), iter, error
      !$acc update self(A)
      do i=1,n 
        do j=1,m 
          write(*,'(f10.2)', advance="no"), A(i,j) 
        enddo 
      enddo
    end if

    iter = iter + 1

  end do
  !$acc end data

  call cpu_time(stop_time) 
  write(*,'(a,f10.3,a)')  ' completed in ', stop_time-start_time, ' seconds'

  deallocate (A,Anew)
end program jacobi

Overwriting jacobi.f90


In [ ]:
!  make clean && make laplace_update && ./laplace_update 10 10

rm -f *.o laplace laplace_*
nvfortran -fast -ta=tesla -Minfo=accel -o laplace_update laplace2d.f90 jacobi.f90
laplace2d.f90:
calcnext:
     34, Generating NVIDIA GPU code
         35, !$acc loop gang ! blockidx%x
             Generating reduction(max:error)
         36, !$acc loop vector(128) ! threadidx%x
     34, Generating implicit copyin(a(:n-1,:m-1)) [if not already present]
         Generating implicit copy(error) [if not already present]
         Generating implicit copyout(anew(1:n-2,1:m-2)) [if not already present]
     36, Loop is parallelizable
swap:
     52, Generating NVIDIA GPU code
         53, !$acc loop gang ! blockidx%x
         54, !$acc loop vector(128) ! threadidx%x
     52, Generating implicit copyout(a(1:n-2,1:m-2)) [if not already present]
         Generating implicit copyin(anew(1:n-2,1:m-2)) [if not already present]
     54, Loop is parallelizable
jacobi.f90:
jacobi:
     23, Generating copyin(a(:,:),anew(:,:)) [if not already present]
     31, Generating upda

# Chapter3 profiling

<img src="https://www.nvidia.com/content/dam/en-zz/Solutions/about-nvidia/logo-and-brand/02-nvidia-logo-color-wht-500x200-4c25-d@2x.png" height="100"></td></tr>
</table>

## CPU profling with gprof

In [ ]:
!mkdir -p /content/lab3 

In [ ]:
cd /content/lab3 

/content/lab3


In [ ]:
%%file test_gprof.c
#include<stdio.h>


void subroutine_c(void)
{
    printf("\n   Inside subroutine_c()\n");
    int i = 0;

    for(;i<200000000;i++);

    return;
}


void subroutine_d(void)
{
    printf("\n   Inside subroutine_d()\n");
    int i = 0;

    for(;i<800000000;i++);

    return;
}



void func1(void)
{
    printf("\n Inside func1 \n");
    int i = 0;

    for(;i<400000000;i++);
 
    subroutine_c();
    subroutine_c(); 

    return;
}

static void func2(void)
{
    printf("\n Inside func2 \n");
    int i = 0;

    for(;i<800000000;i++);
    subroutine_d(); 
    return;
}


int main(void)
{
    printf("\n Inside main()\n");
    int i = 0;

    for(;i<800000000;i++);
    func1();
    func2();

    return 0;
}

Writing test_gprof.c


In [ ]:
!nvc -pg test_gprof.c -o a.out_profile

In [ ]:
!./a.out_profile


 Inside main()

 Inside func1 

   Inside subroutine_c()

   Inside subroutine_c()

 Inside func2 

   Inside subroutine_d()


In [ ]:
!ls -alh gmon.out

-rw-r--r-- 1 root root 713 Sep  1 08:13 gmon.out


In [ ]:
!gprof a.out_profile gmon.out > result.txt

In [ ]:
!cat result.txt

Flat profile:

Each sample counts as 0.01 seconds.
  %   cumulative   self              self     total           
 time   seconds   seconds    calls   s/call   s/call  name    
 25.13      1.56     1.56        1     1.56     1.56  subroutine_d
 25.13      3.11     1.56                             main
 24.97      4.66     1.55        1     1.55     3.10  func2
 12.65      5.44     0.78        2     0.39     0.39  subroutine_c
 12.48      6.21     0.77        1     0.77     1.56  func1

 %         the percentage of the total running time of the
time       program used by this function.

cumulative a running sum of the number of seconds accounted
 seconds   for by this function and those listed above it.

 self      the number of seconds accounted for by this
seconds    function alone.  This is the major sort for this
           listing.

calls      the number of times this function was invoked, if
           this function is profiled, else blank.

 self      the average number of millis

## nvtx for host  code profile in C/C++

In [ ]:
%%file test_gprof.c
#include<stdio.h>
#include <nvtx3/nvToolsExt.h>


void subroutine_c(void)
{
    printf("\n   Inside subroutine_c()\n");
    int i = 0;

    for(;i<200000000;i++);

    return;
}


void subroutine_d(void)
{
    printf("\n   Inside subroutine_d()\n");
    int i = 0;

    for(;i<800000000;i++);

    return;
}



void func1(void)
{
    printf("\n Inside func1 \n");
    int i = 0;

    for(;i<400000000;i++);
 
    subroutine_c();
    subroutine_c(); 

    return;
}

static void func2(void)
{
    printf("\n Inside func2 \n");
    int i = 0;

    for(;i<800000000;i++);
    subroutine_d(); 
    return;
}


int main(void)
{
    printf("\n Inside main()\n");
    int i = 0;
    nvtxRangePush("before"); 
    for(;i<800000000;i++);
    nvtxRangePop(); 
 
     nvtxRangePushA("fucn1");
    func1();
     nvtxRangePop(); 
 
    nvtxRangePushA("fucn2");
    func2();
    nvtxRangePop();  

    return 0;
}

Overwriting test_gprof.c


In [ ]:
!nvc  test_gprof.c -o a.out_profile -I/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/cuda/11.7/include  -L/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/cuda/11.7/lib64 -lnvToolsExt

In [ ]:
!nsys profile -t nvtx  --stats=true ./a.out_profile


 Inside main()

 Inside func1 

   Inside subroutine_c()

   Inside subroutine_c()

 Inside func2 

   Inside subroutine_d()
Generating '/tmp/nsys-report-2031.qdstrm'
[1/3] [========================100%] report2.nsys-rep
[2/3] [========================100%] report2.sqlite
[3/3] Executing 'nvtxsum' stats report

NVTX Range Statistics:

 Time (%)  Total Time (ns)  Instances     Avg (ns)         Med (ns)        Min (ns)       Max (ns)     StdDev (ns)   Style   Range 
 --------  ---------------  ---------  ---------------  ---------------  -------------  -------------  -----------  -------  ------
     50.0    3,188,182,902          1  3,188,182,902.0  3,188,182,902.0  3,188,182,902  3,188,182,902          0.0  PushPop  fucn2 
     25.0    1,594,219,093          1  1,594,219,093.0  1,594,219,093.0  1,594,219,093  1,594,219,093          0.0  PushPop  fucn1 
     25.0    1,592,995,529          1  1,592,995,529.0  1,592,995,529.0  1,592,995,529  1,592,995,529          0.0  PushPop  before

G

## nvtx for fortran 

In [ ]:
%%file nvtx.f90

module nvtx

use iso_c_binding
implicit none

integer,private :: col(7) = [ Z'0000ff00', Z'000000ff', Z'00ffff00', Z'00ff00ff', Z'0000ffff', Z'00ff0000', Z'00ffffff']
character,private,target :: tempName(256)

type, bind(C):: nvtxEventAttributes
  integer(C_INT16_T):: version=1
  integer(C_INT16_T):: size=48 !
  integer(C_INT):: category=0
  integer(C_INT):: colorType=1 ! NVTX_COLOR_ARGB = 1
  integer(C_INT):: color
  integer(C_INT):: payloadType=0 ! NVTX_PAYLOAD_UNKNOWN = 0
  integer(C_INT):: reserved0
  integer(C_INT64_T):: payload   ! union uint,int,double
  integer(C_INT):: messageType=1  ! NVTX_MESSAGE_TYPE_ASCII     = 1 
  type(C_PTR):: message  ! ascii char
end type

interface nvtxRangePush
  ! push range with custom label and standard color
  subroutine nvtxRangePushA(name) bind(C, name='nvtxRangePushA')
  use iso_c_binding
  character(kind=C_CHAR) :: name(256)
  end subroutine

  ! push range with custom label and custom color
  subroutine nvtxRangePushEx(event) bind(C, name='nvtxRangePushEx')
  use iso_c_binding
  import:: nvtxEventAttributes
  type(nvtxEventAttributes):: event
  end subroutine
end interface

interface nvtxRangePop
  subroutine nvtxRangePop() bind(C, name='nvtxRangePop')
  end subroutine
end interface

contains

subroutine nvtxStartRange(name,id)
  character(kind=c_char,len=*) :: name
  integer, optional:: id
  type(nvtxEventAttributes):: event
  character(kind=c_char,len=256) :: trimmed_name
  integer:: i

  trimmed_name=trim(name)//c_null_char

  ! move scalar trimmed_name into character array tempName
  do i=1,LEN(trim(name)) + 1
     tempName(i) = trimmed_name(i:i)
  enddo


  if ( .not. present(id)) then
    call nvtxRangePush(tempName)
  else
    event%color=col(mod(id,7)+1)
    event%message=c_loc(tempName)
    call nvtxRangePushEx(event)
  end if
end subroutine

subroutine nvtxEndRange
  call nvtxRangePop
end subroutine

end module nvtx

Writing nvtx.f90


In [ ]:
%%file main.f90
program main
  use nvtx
  character(len=4) :: itcount

  ! First range with standard color
  call nvtxStartRange("First label")

  do n=1,14
    ! Create custom label for each marker
    write(itcount,'(i4)') n

    ! Range with custom  color
    call nvtxStartRange("Label "//itcount,n)

    print *,"Generate label",n
    ! Add sleep to make markers big 
    call sleep(1)

    call nvtxEndRange
  end do

  call nvtxEndRange
end program main

Writing main.f90


In [ ]:
%%file Makefile

all: nvtx_example

FC=pgf90
#FC=gfortran

nvtx_example:: main.f90 nvtx.f90
	$(FC) -o nvtx_example nvtx.f90 main.f90 -L/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/cuda/11.7/lib64  -lnvToolsExt

clean:
	rm  nvtx_example main.o nvtx.o   nvtx.mod

Overwriting Makefile


In [ ]:
!make  nvtx_example && ./nvtx_example

make: 'nvtx_example' is up to date.
 Generate label            1
 Generate label            2
 Generate label            3
 Generate label            4
 Generate label            5
 Generate label            6
 Generate label            7
 Generate label            8
 Generate label            9
 Generate label           10
 Generate label           11
 Generate label           12
 Generate label           13
 Generate label           14


In [ ]:
!nsys profile -t nvtx --stats=true   ./nvtx_example

 Generate label            1
 Generate label            2
 Generate label            3
 Generate label            4
 Generate label            5
 Generate label            6
 Generate label            7
 Generate label            8
 Generate label            9
 Generate label           10
 Generate label           11
 Generate label           12
 Generate label           13
 Generate label           14
Generating '/tmp/nsys-report-6fe0.qdstrm'
[1/3] [========================100%] report3.nsys-rep
[2/3] [========================100%] report3.sqlite
[3/3] Executing 'nvtxsum' stats report

NVTX Range Statistics:

 Time (%)  Total Time (ns)  Instances      Avg (ns)          Med (ns)         Min (ns)        Max (ns)     StdDev (ns)   Style      Range   
 --------  ---------------  ---------  ----------------  ----------------  --------------  --------------  -----------  -------  -----------
     50.0   14,002,437,519          1  14,002,437,519.0  14,002,437,519.0  14,002,437,519  14,002,43

<img src="https://github.com/openhackathons-org/gpubootcamp/raw/5c077a6c70989492f4bd850240fdcaf8b16fd555/hpc/openacc/English/Fortran/jupyter_notebook/images/development-cycle.png" width=600>

## prepare baseline code

In [ ]:
%%file Makefile

FC := nvfortran
ACCFLAGS:= -fast -ta=tesla -Minfo=accel

laplace: laplace2d.f90 jacobi.f90
	${FC} ${ACCFLAGS} -o laplace laplace2d.f90 jacobi.f90

clean:
	rm -f *.o laplace 

Overwriting Makefile


In [ ]:
%%file jacobi.f90


program jacobi
  use laplace2d
  implicit none
  integer, parameter :: fp_kind=kind(1.0d0)
  integer, parameter :: n=4096, m=4096, iter_max=1000
  integer :: i, j, iter
  real(fp_kind), dimension (:,:), allocatable :: A, Anew
  real(fp_kind) :: tol=1.0e-6_fp_kind, error=1.0_fp_kind
  real(fp_kind) :: start_time, stop_time

  ! allocate ( A(0:n-1,0:m-1), Anew(0:n-1,0:m-1) )

  ! A    = 0.0_fp_kind
  ! Anew = 0.0_fp_kind

  ! Set B.C.
  ! A(0,:)    = 1.0_fp_kind
  ! Anew(0,:) = 1.0_fp_kind
  
  call initialize(A, Anew, m, n)
   
  write(*,'(a,i5,a,i5,a)') 'Jacobi relaxation Calculation:', n, ' x', m, ' mesh'
 
  call cpu_time(start_time) 

  iter=0
  
  !$acc data copy(A) create(Anew)
  do while ( error .gt. tol .and. iter .lt. iter_max )

    error = calcNext(A, Anew, m, n)
    call swap(A, Anew, m, n)

    if(mod(iter,100).eq.0 ) write(*,'(i5,f10.6)'), iter, error

    iter = iter + 1

  end do
  !$acc end data

  call cpu_time(stop_time) 
  write(*,'(a,f10.3,a)')  ' completed in ', stop_time-start_time, ' seconds'

  ! deallocate (A,Anew)
end program jacobi

Overwriting jacobi.f90


In [ ]:
%%file laplace2d.f90 
module laplace2d
  public :: initialize
  public :: calcNext
  public :: swap
  public :: dealloc
  contains
    subroutine initialize(A, Anew, m, n)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(out)   :: A(:,:)
      real(fp_kind),allocatable,intent(out)   :: Anew(:,:)
      integer,intent(in)          :: m, n

      allocate ( A(0:n-1,0:m-1), Anew(0:n-1,0:m-1) )

      A    = 0.0_fp_kind
      Anew = 0.0_fp_kind

      A(0,:)    = 1.0_fp_kind
      Anew(0,:) = 1.0_fp_kind
    end subroutine initialize

    function calcNext(A, Anew, m, n)
      integer, parameter          :: fp_kind=kind(1.0d0)
      real(fp_kind),intent(inout) :: A(0:n-1,0:m-1)
      real(fp_kind),intent(inout) :: Anew(0:n-1,0:m-1)
      integer,intent(in)          :: m, n
      integer                     :: i, j
      real(fp_kind)               :: error

      error=0.0_fp_kind

      !$acc parallel loop reduction(max:error) copyin(A) copyout(Anew)
      do j=1,m-2
        do i=1,n-2
          Anew(i,j) = 0.25_fp_kind * ( A(i+1,j  ) + A(i-1,j  ) + &
                                       A(i  ,j-1) + A(i  ,j+1) )
          error = max( error, abs(Anew(i,j)-A(i,j)) )
        end do
      end do
      calcNext = error
    end function calcNext

    subroutine swap(A, Anew, m, n)
      integer, parameter        :: fp_kind=kind(1.0d0)
      real(fp_kind),intent(out) :: A(0:n-1,0:m-1)
      real(fp_kind),intent(in)  :: Anew(0:n-1,0:m-1)
      integer,intent(in)        :: m, n
      integer                   :: i, j

      !$acc parallel loop copyin(Anew) copyout(A)
      do j=1,m-2
        do i=1,n-2
          A(i,j) = Anew(i,j)
        end do
      end do
    end subroutine swap

    subroutine dealloc(A, Anew)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(in) :: A
      real(fp_kind),allocatable,intent(in) :: Anew
	  
	  deallocate (A,Anew)
    end subroutine
end module laplace2d

Writing laplace2d.f90


In [ ]:
!make clean && make && ./laplace

rm -f *.o laplace 
nvfortran -fast -ta=tesla -Minfo=accel -o laplace laplace2d.f90 jacobi.f90
laplace2d.f90:
calcnext:
     32, Generating copyin(a(:,:)) [if not already present]
         Generating NVIDIA GPU code
         33, !$acc loop gang ! blockidx%x
             Generating reduction(max:error)
         34, !$acc loop vector(128) ! threadidx%x
     32, Generating implicit copy(error) [if not already present]
         Generating copyout(anew(:,:)) [if not already present]
     34, Loop is parallelizable
swap:
     50, Generating copyout(a(:,:)) [if not already present]
         Generating copyin(anew(:,:)) [if not already present]
         Generating NVIDIA GPU code
         51, !$acc loop gang ! blockidx%x
         52, !$acc loop vector(128) ! threadidx%x
     52, Loop is parallelizable
jacobi.f90:
jacobi:
     30, Generating create(anew(:,:)) [if not already present]
         Generating copy(a(:,:)) [if not already present]
Jacobi relaxation Calculation: 4096 x 4096 mesh
    0  

In [ ]:
!make clean && make &&   PGI_ACC_TIME=1 ./laplace 

rm -f *.o laplace 
nvfortran -fast -ta=tesla -Minfo=accel -o laplace laplace2d.f90 jacobi.f90
laplace2d.f90:
calcnext:
     32, Generating copyin(a(:,:)) [if not already present]
         Generating NVIDIA GPU code
         33, !$acc loop gang ! blockidx%x
             Generating reduction(max:error)
         34, !$acc loop vector(128) ! threadidx%x
     32, Generating implicit copy(error) [if not already present]
         Generating copyout(anew(:,:)) [if not already present]
     34, Loop is parallelizable
swap:
     50, Generating copyout(a(:,:)) [if not already present]
         Generating copyin(anew(:,:)) [if not already present]
         Generating NVIDIA GPU code
         51, !$acc loop gang ! blockidx%x
         52, !$acc loop vector(128) ! threadidx%x
     52, Loop is parallelizable
jacobi.f90:
jacobi:
     30, Generating create(anew(:,:)) [if not already present]
         Generating copy(a(:,:)) [if not already present]
Jacobi relaxation Calculation: 4096 x 4096 mesh
libcupt

In [ ]:
!nsys profile -t openacc,nvtx --stats=true   ./laplace

Jacobi relaxation Calculation: 4096 x 4096 mesh
    0  0.250000
  100  0.002397
  200  0.001204
  300  0.000804
  400  0.000603
  500  0.000483
  600  0.000403
  700  0.000345
  800  0.000302
  900  0.000269
 completed in      1.129 seconds
Generating '/tmp/nsys-report-4f83.qdstrm'
[1/8] [========================100%] report5.nsys-rep
[2/8] [========================100%] report5.sqlite
[3/8] Executing 'nvtxsum' stats report
SKIPPED: /content/lab3/report5.sqlite does not contain NV Tools Extension (NVTX) data.
[4/8] Executing 'cudaapisum' stats report

CUDA API Statistics:

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)      Med (ns)     Min (ns)    Max (ns)   StdDev (ns)          Name        
 --------  ---------------  ---------  ------------  ------------  ----------  ----------  -----------  --------------------
     87.5      474,959,319      6,002      79,133.5       9,531.5       1,139   1,268,399    105,052.0  cuStreamSynchronize 
      4.9       26,818,275          1  26,818

In [ ]:
!ncu   ./laplace

Jacobi relaxation Calculation: 4096 x 4096 mesh
==PROF== Connected to process 6510 (/content/lab3/laplace)
==PROF== Profiling "laplace2d_calcnext_32_gpu" - 0: 0%....50%....100% - 10 passes
==PROF== Profiling "laplace2d_calcnext_32_gpu__red" - 1: 0%....50%....100% - 10 passes
==PROF== Profiling "laplace2d_swap_50_gpu" - 2: 0%....50%....100% - 10 passes
    0  0.250000
==PROF== Profiling "laplace2d_calcnext_32_gpu" - 3: 0%....50%....100% - 10 passes
==PROF== Profiling "laplace2d_calcnext_32_gpu__red" - 4: 0%....50%....100% - 10 passes
==PROF== Profiling "laplace2d_swap_50_gpu" - 5: 0%....50%....100% - 10 passes
==PROF== Profiling "laplace2d_calcnext_32_gpu" - 6: 0%....50%....100% - 10 passes
==PROF== Profiling "laplace2d_calcnext_32_gpu__red" - 7: 0%....50%....100% - 10 passes
==PROF== Profiling "laplace2d_swap_50_gpu" - 8: 0%....50%....100% - 10 passes
==PROF== Profiling "laplace2d_calcnext_32_gpu" - 9: 0%....50%....100% - 10 passes
==PROF== Profiling "laplace2d_calcnext_32_gpu__red" - 

## add nvtx 

In [ ]:
%%file Makefile

FC := nvfortran
ACCFLAGS:= -fast -ta=tesla -Minfo=accel

laplace: laplace2d.f90 jacobi.f90 nvtx.f90
	${FC} ${ACCFLAGS} -o laplace laplace2d.f90 jacobi.f90 nvtx.f90 -L/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/cuda/11.7/lib64  -lnvToolsExt

clean:
	rm -f *.o laplace 

Overwriting Makefile


In [ ]:
%%file nvtx.f90

module nvtx
use iso_c_binding
implicit none

integer,private :: col(7) = [ Z'0000ff00', Z'000000ff', Z'00ffff00', Z'00ff00ff', Z'0000ffff', Z'00ff0000', Z'00ffffff']
character,private,target :: tempName(256)

type, bind(C):: nvtxEventAttributes
  integer(C_INT16_T):: version=1
  integer(C_INT16_T):: size=48 !
  integer(C_INT):: category=0
  integer(C_INT):: colorType=1 ! NVTX_COLOR_ARGB = 1
  integer(C_INT):: color
  integer(C_INT):: payloadType=0 ! NVTX_PAYLOAD_UNKNOWN = 0
  integer(C_INT):: reserved0
  integer(C_INT64_T):: payload   ! union uint,int,double
  integer(C_INT):: messageType=1  ! NVTX_MESSAGE_TYPE_ASCII     = 1 
  type(C_PTR):: message  ! ascii char
end type

interface nvtxRangePush
  ! push range with custom label and standard color
  subroutine nvtxRangePushA(name) bind(C, name='nvtxRangePushA')
  use iso_c_binding
  character(kind=C_CHAR) :: name(256)
  end subroutine

  ! push range with custom label and custom color
  subroutine nvtxRangePushEx(event) bind(C, name='nvtxRangePushEx')
  use iso_c_binding
  import:: nvtxEventAttributes
  type(nvtxEventAttributes):: event
  end subroutine
end interface

interface nvtxRangePop
  subroutine nvtxRangePop() bind(C, name='nvtxRangePop')
  end subroutine
end interface

contains

subroutine nvtxStartRange(name,id)
  character(kind=c_char,len=*) :: name
  integer, optional:: id
  type(nvtxEventAttributes):: event
  character(kind=c_char,len=256) :: trimmed_name
  integer:: i

  trimmed_name=trim(name)//c_null_char

  ! move scalar trimmed_name into character array tempName
  do i=1,LEN(trim(name)) + 1
     tempName(i) = trimmed_name(i:i)
  enddo


  if ( .not. present(id)) then
    call nvtxRangePush(tempName)
  else
    event%color=col(mod(id,7)+1)
    event%message=c_loc(tempName)
    call nvtxRangePushEx(event)
  end if
end subroutine

subroutine nvtxEndRange
  call nvtxRangePop
end subroutine

end module nvtx

Overwriting nvtx.f90


In [ ]:
%%file jacobi.f90


program jacobi
  use laplace2d
  use nvtx
  
  implicit none
  integer, parameter :: fp_kind=kind(1.0d0)
  integer, parameter :: n=4096, m=4096, iter_max=1000
  integer :: i, j, iter
  real(fp_kind), dimension (:,:), allocatable :: A, Anew
  real(fp_kind) :: tol=1.0e-6_fp_kind, error=1.0_fp_kind
  real(fp_kind) :: start_time, stop_time

  ! allocate ( A(0:n-1,0:m-1), Anew(0:n-1,0:m-1) )

  ! A    = 0.0_fp_kind
  ! Anew = 0.0_fp_kind

  ! Set B.C.
  ! A(0,:)    = 1.0_fp_kind
  ! Anew(0,:) = 1.0_fp_kind
  
  call nvtxStartRange("init")
  call initialize(A, Anew, m, n)
  call nvtxEndRange
   
  write(*,'(a,i5,a,i5,a)') 'Jacobi relaxation Calculation:', n, ' x', m, ' mesh'
 
  call cpu_time(start_time) 

  iter=0
  
  call nvtxStartRange("outer loop")
  !$acc data copy(A) create(Anew)
  do while ( error .gt. tol .and. iter .lt. iter_max )

    error = calcNext(A, Anew, m, n)
    call swap(A, Anew, m, n)

    if(mod(iter,100).eq.0 ) write(*,'(i5,f10.6)'), iter, error

    iter = iter + 1

  end do
  !$acc end data
  call nvtxEndRange

  call cpu_time(stop_time) 
  write(*,'(a,f10.3,a)')  ' completed in ', stop_time-start_time, ' seconds'

  ! deallocate (A,Anew)
end program jacobi

Overwriting jacobi.f90


In [ ]:
%%file laplace2d.f90 

module laplace2d
  public :: initialize
  public :: calcNext
  public :: swap
  public :: dealloc
  contains
    subroutine initialize(A, Anew, m, n)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(out)   :: A(:,:)
      real(fp_kind),allocatable,intent(out)   :: Anew(:,:)
      integer,intent(in)          :: m, n

      allocate ( A(0:n-1,0:m-1), Anew(0:n-1,0:m-1) )

      A    = 0.0_fp_kind
      Anew = 0.0_fp_kind

      A(0,:)    = 1.0_fp_kind
      Anew(0,:) = 1.0_fp_kind
    end subroutine initialize

    function calcNext(A, Anew, m, n)
      integer, parameter          :: fp_kind=kind(1.0d0)
      real(fp_kind),intent(inout) :: A(0:n-1,0:m-1)
      real(fp_kind),intent(inout) :: Anew(0:n-1,0:m-1)
      integer,intent(in)          :: m, n
      integer                     :: i, j
      real(fp_kind)               :: error

      error=0.0_fp_kind

      !$acc parallel loop reduction(max:error) copyin(A) copyout(Anew)
      do j=1,m-2
        do i=1,n-2
          Anew(i,j) = 0.25_fp_kind * ( A(i+1,j  ) + A(i-1,j  ) + &
                                       A(i  ,j-1) + A(i  ,j+1) )
          error = max( error, abs(Anew(i,j)-A(i,j)) )
        end do
      end do
      calcNext = error
    end function calcNext

    subroutine swap(A, Anew, m, n)
      integer, parameter        :: fp_kind=kind(1.0d0)
      real(fp_kind),intent(out) :: A(0:n-1,0:m-1)
      real(fp_kind),intent(in)  :: Anew(0:n-1,0:m-1)
      integer,intent(in)        :: m, n
      integer                   :: i, j

      !$acc parallel loop copyin(Anew) copyout(A)
      do j=1,m-2
        do i=1,n-2
          A(i,j) = Anew(i,j)
        end do
      end do
    end subroutine swap

    subroutine dealloc(A, Anew)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(in) :: A
      real(fp_kind),allocatable,intent(in) :: Anew
	  
	  deallocate (A,Anew)
    end subroutine
end module laplace2d

Overwriting laplace2d.f90


In [ ]:
!make clean && make &&  ./laplace 

rm -f *.o laplace 
nvfortran -fast -ta=tesla -Minfo=accel -o laplace laplace2d.f90 jacobi.f90 nvtx.f90 -L/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/cuda/11.7/lib64  -lnvToolsExt
laplace2d.f90:
calcnext:
     33, Generating copyin(a(:,:)) [if not already present]
         Generating NVIDIA GPU code
         34, !$acc loop gang ! blockidx%x
             Generating reduction(max:error)
         35, !$acc loop vector(128) ! threadidx%x
     33, Generating implicit copy(error) [if not already present]
         Generating copyout(anew(:,:)) [if not already present]
     35, Loop is parallelizable
swap:
     51, Generating copyout(a(:,:)) [if not already present]
         Generating copyin(anew(:,:)) [if not already present]
         Generating NVIDIA GPU code
         52, !$acc loop gang ! blockidx%x
         53, !$acc loop vector(128) ! threadidx%x
     53, Loop is parallelizable
jacobi.f90:
jacobi:
     34, Generating create(anew(:,:)) [if not already present]
         Generating copy(a(:,:)) [

In [ ]:
!nsys profile -t openacc,nvtx --stats=true   ./laplace

Jacobi relaxation Calculation: 4096 x 4096 mesh
    0  0.250000
  100  0.002397
  200  0.001204
  300  0.000804
  400  0.000603
  500  0.000483
  600  0.000403
  700  0.000345
  800  0.000302
  900  0.000269
 completed in      1.127 seconds
Generating '/tmp/nsys-report-a64f.qdstrm'
[1/8] [========================100%] report6.nsys-rep
[2/8] [========================100%] report6.sqlite
[3/8] Executing 'nvtxsum' stats report

NVTX Range Statistics:

 Time (%)  Total Time (ns)  Instances     Avg (ns)         Med (ns)        Min (ns)       Max (ns)     StdDev (ns)   Style     Range   
 --------  ---------------  ---------  ---------------  ---------------  -------------  -------------  -----------  -------  ----------
     90.8    1,127,155,942          1  1,127,155,942.0  1,127,155,942.0  1,127,155,942  1,127,155,942          0.0  PushPop  outer loop
      9.2      114,647,348          1    114,647,348.0    114,647,348.0    114,647,348    114,647,348          0.0  PushPop  init      

[4

In [ ]:
%%file jacobi.f90


program jacobi
  use laplace2d
  use nvtx
  
  implicit none
  integer, parameter :: fp_kind=kind(1.0d0)
  integer, parameter :: n=4096, m=4096, iter_max=1000
  integer :: i, j, iter
  real(fp_kind), dimension (:,:), allocatable :: A, Anew
  real(fp_kind) :: tol=1.0e-6_fp_kind, error=1.0_fp_kind
  real(fp_kind) :: start_time, stop_time

  ! allocate ( A(0:n-1,0:m-1), Anew(0:n-1,0:m-1) )

  ! A    = 0.0_fp_kind
  ! Anew = 0.0_fp_kind

  ! Set B.C.
  ! A(0,:)    = 1.0_fp_kind
  ! Anew(0,:) = 1.0_fp_kind
  

  call initialize(A, Anew, m, n)

   
  write(*,'(a,i5,a,i5,a)') 'Jacobi relaxation Calculation:', n, ' x', m, ' mesh'
 
  call cpu_time(start_time) 

  iter=0
  

  !$acc data copy(A) create(Anew)
  do while ( error .gt. tol .and. iter .lt. iter_max )

    call nvtxStartRange("calcNext ")
    error = calcNext(A, Anew, m, n)
    call nvtxEndRange

    call nvtxStartRange("swap ")
    call swap(A, Anew, m, n)
    call nvtxEndRange

    if(mod(iter,100).eq.0 ) write(*,'(i5,f10.6)'), iter, error

    iter = iter + 1

  end do
  !$acc end data


  call cpu_time(stop_time) 
  write(*,'(a,f10.3,a)')  ' completed in ', stop_time-start_time, ' seconds'

  ! deallocate (A,Anew)
end program jacobi

Overwriting jacobi.f90


In [ ]:
!make clean && make 

rm -f *.o laplace 
nvfortran -fast -ta=tesla -Minfo=accel -o laplace laplace2d.f90 jacobi.f90 nvtx.f90 -L/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/cuda/11.7/lib64  -lnvToolsExt
laplace2d.f90:
calcnext:
     35, Generating copyin(a(:,:)) [if not already present]
         Generating NVIDIA GPU code
         36, !$acc loop gang ! blockidx%x
             Generating reduction(max:error)
         37, !$acc loop vector(128) ! threadidx%x
     35, Generating implicit copy(error) [if not already present]
         Generating copyout(anew(:,:)) [if not already present]
     37, Loop is parallelizable
swap:
     55, Generating copyout(a(:,:)) [if not already present]
         Generating copyin(anew(:,:)) [if not already present]
         Generating NVIDIA GPU code
         56, !$acc loop gang ! blockidx%x
         57, !$acc loop vector(128) ! threadidx%x
     57, Loop is parallelizable
jacobi.f90:
jacobi:
     35, Generating create(anew(:,:)) [if not already present]
         Generating copy(a(:,:)) [

In [ ]:
!nsys profile -t nvtx --stats=true   ./laplace

Jacobi relaxation Calculation: 4096 x 4096 mesh
    0  0.250000
  100  0.002397
  200  0.001204
  300  0.000804
  400  0.000603
  500  0.000483
  600  0.000403
  700  0.000345
  800  0.000302
  900  0.000269
 completed in      0.705 seconds
Generating '/tmp/nsys-report-ee6c.qdstrm'
[1/3] [========================100%] report12.nsys-rep
[2/3] [========================100%] report12.sqlite
[3/3] Executing 'nvtxsum' stats report

NVTX Range Statistics:

 Time (%)  Total Time (ns)  Instances  Avg (ns)   Med (ns)   Min (ns)  Max (ns)   StdDev (ns)   Style          Range       
 --------  ---------------  ---------  ---------  ---------  --------  ---------  -----------  -------  -------------------
     27.6      275,195,434      1,000  275,195.4  271,358.0   264,175  2,491,952     70,558.2  PushPop  calcNext           
     27.3      271,923,912      1,000  271,923.9  269,720.0   263,551    397,148      7,107.8  PushPop  calcNext inner loop
     22.6      225,666,721      1,000  225,666.7 

In [ ]:
%%file laplace2d.f90 

module laplace2d
  use nvtx 
  public :: initialize
  public :: calcNext
  public :: swap
  public :: dealloc
  contains
    subroutine initialize(A, Anew, m, n)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(out)   :: A(:,:)
      real(fp_kind),allocatable,intent(out)   :: Anew(:,:)
      integer,intent(in)          :: m, n

      allocate ( A(0:n-1,0:m-1), Anew(0:n-1,0:m-1) )

      A    = 0.0_fp_kind
      Anew = 0.0_fp_kind

      A(0,:)    = 1.0_fp_kind
      Anew(0,:) = 1.0_fp_kind
    end subroutine initialize

    function calcNext(A, Anew, m, n)
      integer, parameter          :: fp_kind=kind(1.0d0)
      real(fp_kind),intent(inout) :: A(0:n-1,0:m-1)
      real(fp_kind),intent(inout) :: Anew(0:n-1,0:m-1)
      integer,intent(in)          :: m, n
      integer                     :: i, j
      real(fp_kind)               :: error

      error=0.0_fp_kind
      call nvtxStartRange("calcNext inner loop " )

      !$acc parallel loop reduction(max:error) copyin(A) copyout(Anew)
      do j=1,m-2
        do i=1,n-2
          Anew(i,j) = 0.25_fp_kind * ( A(i+1,j  ) + A(i-1,j  ) + &
                                       A(i  ,j-1) + A(i  ,j+1) )
          error = max( error, abs(Anew(i,j)-A(i,j)) )
        end do
      end do
      call nvtxEndRange
      calcNext = error
    end function calcNext

    subroutine swap(A, Anew, m, n)
      integer, parameter        :: fp_kind=kind(1.0d0)
      real(fp_kind),intent(out) :: A(0:n-1,0:m-1)
      real(fp_kind),intent(in)  :: Anew(0:n-1,0:m-1)
      integer,intent(in)        :: m, n
      integer                   :: i, j

      call nvtxStartRange("swap inner loop " )
      !$acc parallel loop copyin(Anew) copyout(A)
      do j=1,m-2
        do i=1,n-2
          A(i,j) = Anew(i,j)
        end do
      end do
    call nvtxEndRange  
    end subroutine swap

    subroutine dealloc(A, Anew)
      integer, parameter :: fp_kind=kind(1.0d0)
      real(fp_kind),allocatable,intent(in) :: A
      real(fp_kind),allocatable,intent(in) :: Anew
	  
	  deallocate (A,Anew)
    end subroutine
end module laplace2d

Overwriting laplace2d.f90


In [ ]:
!make clean && make 

rm -f *.o laplace 
nvfortran -fast -ta=tesla -Minfo=accel -o laplace laplace2d.f90 jacobi.f90 nvtx.f90 -L/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/cuda/11.7/lib64  -lnvToolsExt
laplace2d.f90:
calcnext:
     35, Generating copyin(a(:,:)) [if not already present]
         Generating NVIDIA GPU code
         36, !$acc loop gang ! blockidx%x
             Generating reduction(max:error)
         37, !$acc loop vector(128) ! threadidx%x
     35, Generating implicit copy(error) [if not already present]
         Generating copyout(anew(:,:)) [if not already present]
     37, Loop is parallelizable
swap:
     55, Generating copyout(a(:,:)) [if not already present]
         Generating copyin(anew(:,:)) [if not already present]
         Generating NVIDIA GPU code
         56, !$acc loop gang ! blockidx%x
         57, !$acc loop vector(128) ! threadidx%x
     57, Loop is parallelizable
jacobi.f90:
jacobi:
     35, Generating create(anew(:,:)) [if not already present]
         Generating copy(a(:,:)) [

In [ ]:
!nsys profile -t nvtx --stats=true   ./laplace

Jacobi relaxation Calculation: 4096 x 4096 mesh
    0  0.250000
  100  0.002397
  200  0.001204
  300  0.000804
  400  0.000603
  500  0.000483
  600  0.000403
  700  0.000345
  800  0.000302
  900  0.000269
 completed in      0.706 seconds
Generating '/tmp/nsys-report-b998.qdstrm'
[1/3] [========================100%] report13.nsys-rep
[2/3] [========================100%] report13.sqlite
[3/3] Executing 'nvtxsum' stats report

NVTX Range Statistics:

 Time (%)  Total Time (ns)  Instances  Avg (ns)   Med (ns)   Min (ns)  Max (ns)  StdDev (ns)   Style          Range       
 --------  ---------------  ---------  ---------  ---------  --------  --------  -----------  -------  -------------------
     27.5      274,106,791      1,000  274,106.8  273,704.0   264,795   421,717      8,095.8  PushPop  calcNext           
     27.4      273,156,593      1,000  273,156.6  272,745.5   264,054   417,450      7,870.5  PushPop  calcNext inner loop
     22.6      225,161,800      1,000  225,161.8  224

In [ ]:
%%file jacobi.f90


program jacobi
  use laplace2d
  use nvtx
  
  implicit none
  integer, parameter :: fp_kind=kind(1.0d0)
  integer, parameter :: n=4096, m=4096, iter_max=1000
  integer :: i, j, iter
  real(fp_kind), dimension (:,:), allocatable :: A, Anew
  real(fp_kind) :: tol=1.0e-6_fp_kind, error=1.0_fp_kind
  real(fp_kind) :: start_time, stop_time

  ! allocate ( A(0:n-1,0:m-1), Anew(0:n-1,0:m-1) )

  ! A    = 0.0_fp_kind
  ! Anew = 0.0_fp_kind

  ! Set B.C.
  ! A(0,:)    = 1.0_fp_kind
  ! Anew(0,:) = 1.0_fp_kind
  

  call initialize(A, Anew, m, n)

   
  write(*,'(a,i5,a,i5,a)') 'Jacobi relaxation Calculation:', n, ' x', m, ' mesh'
 
  call cpu_time(start_time) 

  iter=0
  

  !$acc data copy(A) create(Anew)
  do while ( error .gt. tol .and. iter .lt. iter_max )

 
    error = calcNext(A, Anew, m, n)
 
 
    call swap(A, Anew, m, n)
 

    if(mod(iter,100).eq.0 ) write(*,'(i5,f10.6)'), iter, error

    iter = iter + 1

  end do
  !$acc end data


  call cpu_time(stop_time) 
  write(*,'(a,f10.3,a)')  ' completed in ', stop_time-start_time, ' seconds'

  ! deallocate (A,Anew)
end program jacobi

Overwriting jacobi.f90


In [ ]:
!make clean && make  

rm -f *.o laplace 
nvfortran -fast -ta=tesla -Minfo=accel -o laplace laplace2d.f90 jacobi.f90 nvtx.f90 -L/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/cuda/11.7/lib64  -lnvToolsExt
laplace2d.f90:
calcnext:
     35, Generating copyin(a(:,:)) [if not already present]
         Generating NVIDIA GPU code
         36, !$acc loop gang ! blockidx%x
             Generating reduction(max:error)
         37, !$acc loop vector(128) ! threadidx%x
     35, Generating implicit copy(error) [if not already present]
         Generating copyout(anew(:,:)) [if not already present]
     37, Loop is parallelizable
swap:
     55, Generating copyout(a(:,:)) [if not already present]
         Generating copyin(anew(:,:)) [if not already present]
         Generating NVIDIA GPU code
         56, !$acc loop gang ! blockidx%x
         57, !$acc loop vector(128) ! threadidx%x
     57, Loop is parallelizable
jacobi.f90:
jacobi:
     35, Generating create(anew(:,:)) [if not already present]
         Generating copy(a(:,:)) [

In [ ]:
!nsys profile -t nvtx --stats=true   ./laplace

Jacobi relaxation Calculation: 4096 x 4096 mesh
    0  0.250000
  100  0.002397
  200  0.001204
  300  0.000804
  400  0.000603
  500  0.000483
  600  0.000403
  700  0.000345
  800  0.000302
  900  0.000269
 completed in      0.703 seconds
Generating '/tmp/nsys-report-a571.qdstrm'
[1/3] [========================100%] report14.nsys-rep
[2/3] [========================100%] report14.sqlite
[3/3] Executing 'nvtxsum' stats report

NVTX Range Statistics:

 Time (%)  Total Time (ns)  Instances  Avg (ns)   Med (ns)   Min (ns)  Max (ns)  StdDev (ns)   Style          Range       
 --------  ---------------  ---------  ---------  ---------  --------  --------  -----------  -------  -------------------
     54.9      273,138,146      1,000  273,138.1  273,568.5   264,059   414,084      7,505.8  PushPop  calcNext inner loop
     45.1      224,000,825      1,000  224,000.8  223,370.0   219,577   254,441      2,681.7  PushPop  swap inner loop    

Generated:
    /content/lab3/report14.nsys-rep
    /

## Optimize Loop Schedules
The compiler has analyzed the loops in our two main functions and scheduled the iterations of the loops to run in parallel on our GPU and Multicore CPU. The compiler is usually pretty good at choosing how to break up loop iterations to run well on parallel accelerators, but sometimes we can eke out just a little more performance by guiding the compiler to make specific choices. First, let's look at the choices the compiler made for us. We'll focus on the calcNext routine, but you should look at the swap routine too. Here's the compiler feedback for that routine:
```
calcnext:
     58, Generating copyin(a(:,:))
         Accelerator kernel generated
         Generating Tesla code
         58, Generating reduction(max:error)
         59, !$acc loop gang ! blockidx%x
         60, !$acc loop vector(128) ! threadidx%x
     58, Generating implicit copy(error)
         Generating copyout(anew(:,:))
     60, Loop is parallelizable
```     
The main loops on interest in calcNext are on lines 59 and 60. I see that the compiler has told me what loop clauses it chose for each of those loops. The outermost loop is treated as a gang loop, meaning it broke that loop up into chunks that can be spread out across the GPU or CPU cores easily. If you have programmed in CUDA before, you'll recognize that the compiler is mapping this loop to the CUDA thread blocks. The innermost loop is mapped instead to vector parallelism. You can think of a vector as some number of data cells that get the same operation applied to them at the same time. On any modern processor technology you need this mixture of coarse grained and fine grained parallelism to effectively use the hardware. Vector (fine grained) parallelism can operate extremely efficiently when performing the same operation on a bunch of data, but there's limits to how long a vector you can build. Gang (coarse grained) parallelism is highly scalable, because each chunk of work can operate completely independently of each other chunk, making it ideal for allowing processor cores to operate independently of each other.

Let's look at some loop clauses that allow you to tune how the compiler maps our loop iterations to these different types of parallelism.

## Collapse Clause
The collapse clause allows us to transform a multi-dimensional loop nest into a single-dimensional loop. This process is helpful for increasing the overall length (which usually increases parallelism) of our loops, and will often help with memory locality. Let's look at the syntax.

```
!$acc parallel loop collapse( N )
```

Where N is the number of loops to collapse.

```
!$acc parallel loop collapse( 3 )
do i = 1, N
    do j = 1, M
        do k = 1, Q
            < loop code >
        end do
    end do
end do
```
This code will combine the 3-dimensional loop nest into a single 1-dimensional loop. The loops in our example code are fairly long-running, so I don't expect a lot of speed-up from collapsing them together, but let's try it anyway.

Implementing the Collapse Clause